In [1]:
import os
import pickle
import string
import argparse

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [ ]:
# %load NEW03OT-MakeConcordance-Leviticus.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""

from BibleWordList2 import *

BUILDAWIP = 1
BUILDNWIV = 1
BUILDPNIP = 1
BUILDNWIC = 1
BUILDNLIW = 1
BUILDBPIP = 1

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

Leviticus01 = [
    ['Leviticus','Lev','01','001','1',"And the LORD called unto Moses, and spake unto him out of the tabernacle of the congregation, saying,"],
['Leviticus','Lev','01','002','2',"Speak unto the children of Israel, and say unto them, If any man of you bring an offering unto the LORD, ye shall bring your offering of the cattle, \emph{even} of the herd, and of the flock."],
['Leviticus','Lev','01','003','3',"If his offering \emph{be} a burnt sacrifice of the herd, let him offer a male without blemish: he shall offer it of his own voluntary will at the door of the tabernacle of the congregation before the LORD."],
['Leviticus','Lev','01','004','4',"And he shall put his hand upon the head of the burnt offering; and it shall be accepted for him to make atonement for him."],
['Leviticus','Lev','01','005','5',"And he shall kill the bullock before the LORD: and the priests, Aaron's sons, shall bring the blood, and sprinkle the blood round about upon the altar that \emph{is} \emph{by} the door of the tabernacle of the congregation."],
['Leviticus','Lev','01','006','6',"And he shall flay the burnt offering, and cut it into his pieces."],
['Leviticus','Lev','01','007','7',"And the sons of Aaron the priest shall put fire upon the altar, and lay the wood in order upon the fire:"],
['Leviticus','Lev','01','008','8',"And the priests, Aaron's sons, shall lay the parts, the head, and the fat, in order upon the wood that \emph{is} on the fire which \emph{is} upon the altar:"],
['Leviticus','Lev','01','009','9',"But his inwards and his legs shall he wash in water: and the priest shall burn all on the altar, \emph{to} \emph{be} a burnt sacrifice, an offering made by fire, of a sweet savour unto the LORD."],
['Leviticus','Lev','01','010','10',"And if his offering \emph{be} of the flocks, \emph{namely}, of the sheep, or of the goats, for a burnt sacrifice; he shall bring it a male without blemish."],
['Leviticus','Lev','01','011','11',"And he shall kill it on the side of the altar northward before the LORD: and the priests, Aaron's sons, shall sprinkle his blood round about upon the altar."],
['Leviticus','Lev','01','012','12',"And he shall cut it into his pieces, with his head and his fat: and the priest shall lay them in order on the wood that \emph{is} on the fire which \emph{is} upon the altar:"],
['Leviticus','Lev','01','013','13',"But he shall wash the inwards and the legs with water: and the priest shall bring \emph{it} all, and burn \emph{it} upon the altar: it \emph{is} a burnt sacrifice, an offering made by fire, of a sweet savour unto the LORD."],
['Leviticus','Lev','01','014','14',"And if the burnt sacrifice for his offering to the LORD \emph{be} of fowls, then he shall bring his offering of turtledoves, or of young pigeons."],
['Leviticus','Lev','01','015','15',"And the priest shall bring it unto the altar, and wring off his head, and burn \emph{it} on the altar; and the blood thereof shall be wrung out at the side of the altar:"],
['Leviticus','Lev','01','016','16',"And he shall pluck away his crop with his feathers, and cast it beside the altar on the east part, by the place of the ashes:"],
['Leviticus','Lev','01','017','17',"And he shall cleave it with the wings thereof, \emph{but} shall not divide \emph{it} asunder: and the priest shall burn it upon the altar, upon the wood that \emph{is} upon the fire: it \emph{is} a burnt sacrifice, an offering made by fire, of a sweet savour unto the LORD."]
]

Leviticus02 = [
    ['Leviticus','Lev','02','001','1',"And when any will offer a meat offering unto the LORD, his offering shall be \emph{of} fine flour; and he shall pour oil upon it, and put frankincense thereon:"],
['Leviticus','Lev','02','002','2',"And he shall bring it to Aaron's sons the priests: and he shall take thereout his handful of the flour thereof, and of the oil thereof, with all the frankincense thereof; and the priest shall burn the memorial of it upon the altar, \emph{to} \emph{be} an offering made by fire, of a sweet savour unto the LORD:"],
['Leviticus','Lev','02','003','3',"And the remnant of the meat offering \emph{shall} \emph{be} Aaron's and his sons': \emph{it} \emph{is} a thing most holy of the offerings of the LORD made by fire."],
['Leviticus','Lev','02','004','4',"And if thou bring an oblation of a meat offering baken in the oven, \emph{it} \emph{shall} \emph{be} unleavened cakes of fine flour mingled with oil, or unleavened wafers anointed with oil."],
['Leviticus','Lev','02','005','5',"And if thy oblation \emph{be} a meat offering \emph{baken} in a pan, it shall be \emph{of} fine flour unleavened, mingled with oil."],
['Leviticus','Lev','02','006','6',"Thou shalt part it in pieces, and pour oil thereon: it \emph{is} a meat offering."],
['Leviticus','Lev','02','007','7',"And if thy oblation \emph{be} a meat offering \emph{baken} in the fryingpan, it shall be made \emph{of} fine flour with oil."],
['Leviticus','Lev','02','008','8',"And thou shalt bring the meat offering that is made of these things unto the LORD: and when it is presented unto the priest, he shall bring it unto the altar."],
['Leviticus','Lev','02','009','9',"And the priest shall take from the meat offering a memorial thereof, and shall burn \emph{it} upon the altar: \emph{it} \emph{is} an offering made by fire, of a sweet savour unto the LORD."],
['Leviticus','Lev','02','010','10',"And that which is left of the meat offering \emph{shall} \emph{be} Aaron's and his sons': \emph{it} \emph{is} a thing most holy of the offerings of the LORD made by fire."],
['Leviticus','Lev','02','011','11',"No meat offering, which ye shall bring unto the LORD, shall be made with leaven: for ye shall burn no leaven, nor any honey, in any offering of the LORD made by fire."],
['Leviticus','Lev','02','012','12',"As for the oblation of the firstfruits, ye shall offer them unto the LORD: but they shall not be burnt on the altar for a sweet savour."],
['Leviticus','Lev','02','013','13',"And every oblation of thy meat offering shalt thou season with salt; neither shalt thou suffer the salt of the covenant of thy God to be lacking from thy meat offering: with all thine offerings thou shalt offer salt."],
['Leviticus','Lev','02','014','14',"And if thou offer a meat offering of thy firstfruits unto the LORD, thou shalt offer for the meat offering of thy firstfruits green ears of corn dried by the fire, \emph{even} corn beaten out of full ears."],
['Leviticus','Lev','02','015','15',"And thou shalt put oil upon it, and lay frankincense thereon: it \emph{is} a meat offering."],
['Leviticus','Lev','02','016','16',"And the priest shall burn the memorial of it, \emph{part} of the beaten corn thereof, and \emph{part} of the oil thereof, with all the frankincense thereof: \emph{it} \emph{is} an offering made by fire unto the LORD."]
]

Leviticus03 = [
    ['Leviticus','Lev','03','001','1',"And if his oblation \emph{be} a sacrifice of peace offering, if he offer \emph{it} of the herd; whether \emph{it} \emph{be} a male or female, he shall offer it without blemish before the LORD."],
['Leviticus','Lev','03','002','2',"And he shall lay his hand upon the head of his offering, and kill it \emph{at} the door of the tabernacle of the congregation: and Aaron's sons the priests shall sprinkle the blood upon the altar round about."],
['Leviticus','Lev','03','003','3',"And he shall offer of the sacrifice of the peace offering an offering made by fire unto the LORD; the fat that covereth the inwards, and all the fat that \emph{is} upon the inwards,"],
['Leviticus','Lev','03','004','4',"And the two kidneys, and the fat that \emph{is} on them, which \emph{is} by the flanks, and the caul above the liver, with the kidneys, it shall he take away."],
['Leviticus','Lev','03','005','5',"And Aaron's sons shall burn it on the altar upon the burnt sacrifice, which \emph{is} upon the wood that \emph{is} on the fire: \emph{it} \emph{is} an offering made by fire, of a sweet savour unto the LORD."],
['Leviticus','Lev','03','006','6',"And if his offering for a sacrifice of peace offering unto the LORD \emph{be} of the flock; male or female, he shall offer it without blemish."],
['Leviticus','Lev','03','007','7',"If he offer a lamb for his offering, then shall he offer it before the LORD."],
['Leviticus','Lev','03','008','8',"And he shall lay his hand upon the head of his offering, and kill it before the tabernacle of the congregation: and Aaron's sons shall sprinkle the blood thereof round about upon the altar."],
['Leviticus','Lev','03','009','9',"And he shall offer of the sacrifice of the peace offering an offering made by fire unto the LORD; the fat thereof, \emph{and} the whole rump, it shall he take off hard by the backbone; and the fat that covereth the inwards, and all the fat that \emph{is} upon the inwards,"],
['Leviticus','Lev','03','010','10',"And the two kidneys, and the fat that \emph{is} upon them, which \emph{is} by the flanks, and the caul above the liver, with the kidneys, it shall he take away."],
['Leviticus','Lev','03','011','11',"And the priest shall burn it upon the altar: \emph{it} \emph{is} the food of the offering made by fire unto the LORD."],
['Leviticus','Lev','03','012','12',"And if his offering \emph{be} a goat, then he shall offer it before the LORD."],
['Leviticus','Lev','03','013','13',"And he shall lay his hand upon the head of it, and kill it before the tabernacle of the congregation: and the sons of Aaron shall sprinkle the blood thereof upon the altar round about."],
['Leviticus','Lev','03','014','14',"And he shall offer thereof his offering, \emph{even} an offering made by fire unto the LORD; the fat that covereth the inwards, and all the fat that \emph{is} upon the inwards,"],
['Leviticus','Lev','03','015','15',"And the two kidneys, and the fat that \emph{is} upon them, which \emph{is} by the flanks, and the caul above the liver, with the kidneys, it shall he take away."],
['Leviticus','Lev','03','016','16',"And the priest shall burn them upon the altar: \emph{it} \emph{is} the food of the offering made by fire for a sweet savour: all the fat \emph{is} the LORD'S."],
['Leviticus','Lev','03','017','17',"\emph{It} \emph{shall} \emph{be} a perpetual statute for your generations throughout all your dwellings, that ye eat neither fat nor blood."]
]

Leviticus04 = [
    ['Leviticus','Lev','04','001','1',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','04','002','2',"Speak unto the children of Israel, saying, If a soul shall sin through ignorance against any of the commandments of the LORD \emph{concerning} \emph{things} which ought not to be done, and shall do against any of them:"],
['Leviticus','Lev','04','003','3',"If the priest that is anointed do sin according to the sin of the people; then let him bring for his sin, which he hath sinned, a young bullock without blemish unto the LORD for a sin offering."],
['Leviticus','Lev','04','004','4',"And he shall bring the bullock unto the door of the tabernacle of the congregation before the LORD; and shall lay his hand upon the bullock's head, and kill the bullock before the LORD."],
['Leviticus','Lev','04','005','5',"And the priest that is anointed shall take of the bullock's blood, and bring it to the tabernacle of the congregation:"],
['Leviticus','Lev','04','006','6',"And the priest shall dip his finger in the blood, and sprinkle of the blood seven times before the LORD, before the vail of the sanctuary."],
['Leviticus','Lev','04','007','7',"And the priest shall put \emph{some} of the blood upon the horns of the altar of sweet incense before the LORD, which \emph{is} in the tabernacle of the congregation; and shall pour all the blood of the bullock at the bottom of the altar of the burnt offering, which \emph{is} \emph{at} the door of the tabernacle of the congregation."],
['Leviticus','Lev','04','008','8',"And he shall take off from it all the fat of the bullock for the sin offering; the fat that covereth the inwards, and all the fat that \emph{is} upon the inwards,"],
['Leviticus','Lev','04','009','9',"And the two kidneys, and the fat that \emph{is} upon them, which \emph{is} by the flanks, and the caul above the liver, with the kidneys, it shall he take away,"],
['Leviticus','Lev','04','010','10',"As it was taken off from the bullock of the sacrifice of peace offerings: and the priest shall burn them upon the altar of the burnt offering."],
['Leviticus','Lev','04','011','11',"And the skin of the bullock, and all his flesh, with his head, and with his legs, and his inwards, and his dung,"],
['Leviticus','Lev','04','012','12',"Even the whole bullock shall he carry forth without the camp unto a clean place, where the ashes are poured out, and burn him on the wood with fire: where the ashes are poured out shall he be burnt."],
['Leviticus','Lev','04','013','13',"And if the whole congregation of Israel sin through ignorance, and the thing be hid from the eyes of the assembly, and they have done \emph{somewhat} \emph{against} any of the commandments of the LORD \emph{concerning} \emph{things} which should not be done, and are guilty;"],
['Leviticus','Lev','04','014','14',"When the sin, which they have sinned against it, is known, then the congregation shall offer a young bullock for the sin, and bring him before the tabernacle of the congregation."],
['Leviticus','Lev','04','015','15',"And the elders of the congregation shall lay their hands upon the head of the bullock before the LORD: and the bullock shall be killed before the LORD."],
['Leviticus','Lev','04','016','16',"And the priest that is anointed shall bring of the bullock's blood to the tabernacle of the congregation:"],
['Leviticus','Lev','04','017','17',"And the priest shall dip his finger \emph{in} \emph{some} of the blood, and sprinkle \emph{it} seven times before the LORD, \emph{even} before the vail."],
['Leviticus','Lev','04','018','18',"And he shall put \emph{some} of the blood upon the horns of the altar which \emph{is} before the LORD, that \emph{is} in the tabernacle of the congregation, and shall pour out all the blood at the bottom of the altar of the burnt offering, which \emph{is} \emph{at} the door of the tabernacle of the congregation."],
['Leviticus','Lev','04','019','19',"And he shall take all his fat from him, and burn \emph{it} upon the altar."],
['Leviticus','Lev','04','020','20',"And he shall do with the bullock as he did with the bullock for a sin offering, so shall he do with this: and the priest shall make an atonement for them, and it shall be forgiven them."],
['Leviticus','Lev','04','021','21',"And he shall carry forth the bullock without the camp, and burn him as he burned the first bullock: it \emph{is} a sin offering for the congregation."],
['Leviticus','Lev','04','022','22',"When a ruler hath sinned, and done \emph{somewhat} through ignorance \emph{against} any of the commandments of the LORD his God \emph{concerning} \emph{things} which should not be done, and is guilty;"],
['Leviticus','Lev','04','023','23',"Or if his sin, wherein he hath sinned, come to his knowledge; he shall bring his offering, a kid of the goats, a male without blemish:"],
['Leviticus','Lev','04','024','24',"And he shall lay his hand upon the head of the goat, and kill it in the place where they kill the burnt offering before the LORD: it \emph{is} a sin offering."],
['Leviticus','Lev','04','025','25',"And the priest shall take of the blood of the sin offering with his finger, and put \emph{it} upon the horns of the altar of burnt offering, and shall pour out his blood at the bottom of the altar of burnt offering."],
['Leviticus','Lev','04','026','26',"And he shall burn all his fat upon the altar, as the fat of the sacrifice of peace offerings: and the priest shall make an atonement for him as concerning his sin, and it shall be forgiven him."],
['Leviticus','Lev','04','027','27',"And if any one of the common people sin through ignorance, while he doeth \emph{somewhat} \emph{against} any of the commandments of the LORD \emph{concerning} \emph{things} which ought not to be done, and be guilty;"],
['Leviticus','Lev','04','028','28',"Or if his sin, which he hath sinned, come to his knowledge: then he shall bring his offering, a kid of the goats, a female without blemish, for his sin which he hath sinned."],
['Leviticus','Lev','04','029','29',"And he shall lay his hand upon the head of the sin offering, and slay the sin offering in the place of the burnt offering."],
['Leviticus','Lev','04','030','30',"And the priest shall take of the blood thereof with his finger, and put \emph{it} upon the horns of the altar of burnt offering, and shall pour out all the blood thereof at the bottom of the altar."],
['Leviticus','Lev','04','031','31',"And he shall take away all the fat thereof, as the fat is taken away from off the sacrifice of peace offerings; and the priest shall burn \emph{it} upon the altar for a sweet savour unto the LORD; and the priest shall make an atonement for him, and it shall be forgiven him."],
['Leviticus','Lev','04','032','32',"And if he bring a lamb for a sin offering, he shall bring it a female without blemish."],
['Leviticus','Lev','04','033','33',"And he shall lay his hand upon the head of the sin offering, and slay it for a sin offering in the place where they kill the burnt offering."],
['Leviticus','Lev','04','034','34',"And the priest shall take of the blood of the sin offering with his finger, and put \emph{it} upon the horns of the altar of burnt offering, and shall pour out all the blood thereof at the bottom of the altar:"],
['Leviticus','Lev','04','035','35',"And he shall take away all the fat thereof, as the fat of the lamb is taken away from the sacrifice of the peace offerings; and the priest shall burn them upon the altar, according to the offerings made by fire unto the LORD: and the priest shall make an atonement for his sin that he hath committed, and it shall be forgiven him."]
]

Leviticus05 = [
    ['Leviticus','Lev','05','001','1',"And if a soul sin, and hear the voice of swearing, and \emph{is} a witness, whether he hath seen or known \emph{of} \emph{it}; if he do not utter \emph{it}, then he shall bear his iniquity."],
['Leviticus','Lev','05','002','2',"Or if a soul touch any unclean thing, whether \emph{it} \emph{be} a carcase of an unclean beast, or a carcase of unclean cattle, or the carcase of unclean creeping things, and \emph{if} it be hidden from him; he also shall be unclean, and guilty."],
['Leviticus','Lev','05','003','3',"Or if he touch the uncleanness of man, whatsoever uncleanness \emph{it} \emph{be} that a man shall be defiled withal, and it be hid from him; when he knoweth \emph{of} \emph{it}, then he shall be guilty."],
['Leviticus','Lev','05','004','4',"Or if a soul swear, pronouncing with \emph{his} lips to do evil, or to do good, whatsoever \emph{it} \emph{be} that a man shall pronounce with an oath, and it be hid from him; when he knoweth \emph{of} \emph{it}, then he shall be guilty in one of these."],
['Leviticus','Lev','05','005','5',"And it shall be, when he shall be guilty in one of these \emph{things}, that he shall confess that he hath sinned in that \emph{thing}:"],
['Leviticus','Lev','05','006','6',"And he shall bring his trespass offering unto the LORD for his sin which he hath sinned, a female from the flock, a lamb or a kid of the goats, for a sin offering; and the priest shall make an atonement for him concerning his sin."],
['Leviticus','Lev','05','007','7',"And if he be not able to bring a lamb, then he shall bring for his trespass, which he hath committed, two turtledoves, or two young pigeons, unto the LORD; one for a sin offering, and the other for a burnt offering."],
['Leviticus','Lev','05','008','8',"And he shall bring them unto the priest, who shall offer \emph{that} which \emph{is} for the sin offering first, and wring off his head from his neck, but shall not divide \emph{it} asunder:"],
['Leviticus','Lev','05','009','9',"And he shall sprinkle of the blood of the sin offering upon the side of the altar; and the rest of the blood shall be wrung out at the bottom of the altar: it \emph{is} a sin offering."],
['Leviticus','Lev','05','010','10',"And he shall offer the second \emph{for} a burnt offering, according to the manner: and the priest shall make an atonement for him for his sin which he hath sinned, and it shall be forgiven him."],
['Leviticus','Lev','05','011','11',"But if he be not able to bring two turtledoves, or two young pigeons, then he that sinned shall bring for his offering the tenth part of an ephah of fine flour for a sin offering; he shall put no oil upon it, neither shall he put \emph{any} frankincense thereon: for it \emph{is} a sin offering."],
['Leviticus','Lev','05','012','12',"Then shall he bring it to the priest, and the priest shall take his handful of it, \emph{even} a memorial thereof, and burn \emph{it} on the altar, according to the offerings made by fire unto the LORD: it \emph{is} a sin offering."],
['Leviticus','Lev','05','013','13',"And the priest shall make an atonement for him as touching his sin that he hath sinned in one of these, and it shall be forgiven him: and \emph{the} \emph{remnant} shall be the priest's, as a meat offering."],
['Leviticus','Lev','05','014','14',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','05','015','15',"If a soul commit a trespass, and sin through ignorance, in the holy things of the LORD; then he shall bring for his trespass unto the LORD a ram without blemish out of the flocks, with thy estimation by shekels of silver, after the shekel of the sanctuary, for a trespass offering:"],
['Leviticus','Lev','05','016','16',"And he shall make amends for the harm that he hath done in the holy thing, and shall add the fifth part thereto, and give it unto the priest: and the priest shall make an atonement for him with the ram of the trespass offering, and it shall be forgiven him."],
['Leviticus','Lev','05','017','17',"And if a soul sin, and commit any of these things which are forbidden to be done by the commandments of the LORD; though he wist \emph{it} not, yet is he guilty, and shall bear his iniquity."],
['Leviticus','Lev','05','018','18',"And he shall bring a ram without blemish out of the flock, with thy estimation, for a trespass offering, unto the priest: and the priest shall make an atonement for him concerning his ignorance wherein he erred and wist \emph{it} not, and it shall be forgiven him."],
['Leviticus','Lev','05','019','19',"It \emph{is} a trespass offering: he hath certainly trespassed against the LORD."]
]

Leviticus06 = [
    ['Leviticus','Lev','06','001','1',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','06','002','2',"If a soul sin, and commit a trespass against the LORD, and lie unto his neighbour in that which was delivered him to keep, or in fellowship, or in a thing taken away by violence, or hath deceived his neighbour;"],
['Leviticus','Lev','06','003','3',"Or have found that which was lost, and lieth concerning it, and sweareth falsely; in any of all these that a man doeth, sinning therein:"],
['Leviticus','Lev','06','004','4',"Then it shall be, because he hath sinned, and is guilty, that he shall restore that which he took violently away, or the thing which he hath deceitfully gotten, or that which was delivered him to keep, or the lost thing which he found,"],
['Leviticus','Lev','06','005','5',"Or all that about which he hath sworn falsely; he shall even restore it in the principal, and shall add the fifth part more thereto, \emph{and} give it unto him to whom it appertaineth, in the day of his trespass offering."],
['Leviticus','Lev','06','006','6',"And he shall bring his trespass offering unto the LORD, a ram without blemish out of the flock, with thy estimation, for a trespass offering, unto the priest:"],
['Leviticus','Lev','06','007','7',"And the priest shall make an atonement for him before the LORD: and it shall be forgiven him for any thing of all that he hath done in trespassing therein."],
['Leviticus','Lev','06','008','8',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','06','009','9',"Command Aaron and his sons, saying, This \emph{is} the law of the burnt offering: It \emph{is} the burnt offering, because of the burning upon the altar all night unto the morning, and the fire of the altar shall be burning in it."],
['Leviticus','Lev','06','010','10',"And the priest shall put on his linen garment, and his linen breeches shall he put upon his flesh, and take up the ashes which the fire hath consumed with the burnt offering on the altar, and he shall put them beside the altar."],
['Leviticus','Lev','06','011','11',"And he shall put off his garments, and put on other garments, and carry forth the ashes without the camp unto a clean place."],
['Leviticus','Lev','06','012','12',"And the fire upon the altar shall be burning in it; it shall not be put out: and the priest shall burn wood on it every morning, and lay the burnt offering in order upon it; and he shall burn thereon the fat of the peace offerings."],
['Leviticus','Lev','06','013','13',"The fire shall ever be burning upon the altar; it shall never go out."],
['Leviticus','Lev','06','014','14',"And this \emph{is} the law of the meat offering: the sons of Aaron shall offer it before the LORD, before the altar."],
['Leviticus','Lev','06','015','15',"And he shall take of it his handful, of the flour of the meat offering, and of the oil thereof, and all the frankincense which \emph{is} upon the meat offering, and shall burn \emph{it} upon the altar \emph{for} a sweet savour, \emph{even} the memorial of it, unto the LORD."],
['Leviticus','Lev','06','016','16',"And the remainder thereof shall Aaron and his sons eat: with unleavened bread shall it be eaten in the holy place; in the court of the tabernacle of the congregation they shall eat it."],
['Leviticus','Lev','06','017','17',"It shall not be baken with leaven. I have given it \emph{unto} \emph{them} \emph{for} their portion of my offerings made by fire; it \emph{is} most holy, as \emph{is} the sin offering, and as the trespass offering."],
['Leviticus','Lev','06','018','18',"All the males among the children of Aaron shall eat of it. \emph{It} \emph{shall} \emph{be} a statute for ever in your generations concerning the offerings of the LORD made by fire: every one that toucheth them shall be holy."],
['Leviticus','Lev','06','019','19',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','06','020','20',"This \emph{is} the offering of Aaron and of his sons, which they shall offer unto the LORD in the day when he is anointed; the tenth part of an ephah of fine flour for a meat offering perpetual, half of it in the morning, and half thereof at night."],
['Leviticus','Lev','06','021','21',"In a pan it shall be made with oil; \emph{and} \emph{when} \emph{it} \emph{is} baken, thou shalt bring it in: \emph{and} the baken pieces of the meat offering shalt thou offer \emph{for} a sweet savour unto the LORD."],
['Leviticus','Lev','06','022','22',"And the priest of his sons that is anointed in his stead shall offer it: \emph{it} \emph{is} a statute for ever unto the LORD; it shall be wholly burnt."],
['Leviticus','Lev','06','023','23',"For every meat offering for the priest shall be wholly burnt: it shall not be eaten."],
['Leviticus','Lev','06','024','24',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','06','025','25',"Speak unto Aaron and to his sons, saying, This \emph{is} the law of the sin offering: In the place where the burnt offering is killed shall the sin offering be killed before the LORD: it \emph{is} most holy."],
['Leviticus','Lev','06','026','26',"The priest that offereth it for sin shall eat it: in the holy place shall it be eaten, in the court of the tabernacle of the congregation."],
['Leviticus','Lev','06','027','27',"Whatsoever shall touch the flesh thereof shall be holy: and when there is sprinkled of the blood thereof upon any garment, thou shalt wash that whereon it was sprinkled in the holy place."],
['Leviticus','Lev','06','028','28',"But the earthen vessel wherein it is sodden shall be broken: and if it be sodden in a brasen pot, it shall be both scoured, and rinsed in water."],
['Leviticus','Lev','06','029','29',"All the males among the priests shall eat thereof: it \emph{is} most holy."],
['Leviticus','Lev','06','030','30',"And no sin offering, whereof \emph{any} of the blood is brought into the tabernacle of the congregation to reconcile \emph{withal} in the holy \emph{place}, shall be eaten: it shall be burnt in the fire."]
]

Leviticus07 = [
    ['Leviticus','Lev','07','001','1',"Likewise this \emph{is} the law of the trespass offering: it \emph{is} most holy."],
['Leviticus','Lev','07','002','2',"In the place where they kill the burnt offering shall they kill the trespass offering: and the blood thereof shall he sprinkle round about upon the altar."],
['Leviticus','Lev','07','003','3',"And he shall offer of it all the fat thereof; the rump, and the fat that covereth the inwards,"],
['Leviticus','Lev','07','004','4',"And the two kidneys, and the fat that \emph{is} on them, which \emph{is} by the flanks, and the caul \emph{that} \emph{is} above the liver, with the kidneys, it shall he take away:"],
['Leviticus','Lev','07','005','5',"And the priest shall burn them upon the altar \emph{for} an offering made by fire unto the LORD: it \emph{is} a trespass offering."],
['Leviticus','Lev','07','006','6',"Every male among the priests shall eat thereof: it shall be eaten in the holy place: it \emph{is} most holy."],
['Leviticus','Lev','07','007','7',"As the sin offering \emph{is}, so \emph{is} the trespass offering: \emph{there} \emph{is} one law for them: the priest that maketh atonement therewith shall have \emph{it}."],
['Leviticus','Lev','07','008','8',"And the priest that offereth any man's burnt offering, \emph{even} the priest shall have to himself the skin of the burnt offering which he hath offered."],
['Leviticus','Lev','07','009','9',"And all the meat offering that is baken in the oven, and all that is dressed in the fryingpan, and in the pan, shall be the priest's that offereth it."],
['Leviticus','Lev','07','010','10',"And every meat offering, mingled with oil, and dry, shall all the sons of Aaron have, one \emph{as} \emph{much} as another."],
['Leviticus','Lev','07','011','11',"And this \emph{is} the law of the sacrifice of peace offerings, which he shall offer unto the LORD."],
['Leviticus','Lev','07','012','12',"If he offer it for a thanksgiving, then he shall offer with the sacrifice of thanksgiving unleavened cakes mingled with oil, and unleavened wafers anointed with oil, and cakes mingled with oil, of fine flour, fried."],
['Leviticus','Lev','07','013','13',"Besides the cakes, he shall offer \emph{for} his offering leavened bread with the sacrifice of thanksgiving of his peace offerings."],
['Leviticus','Lev','07','014','14',"And of it he shall offer one out of the whole oblation \emph{for} an heave offering unto the LORD, \emph{and} it shall be the priest's that sprinkleth the blood of the peace offerings."],
['Leviticus','Lev','07','015','15',"And the flesh of the sacrifice of his peace offerings for thanksgiving shall be eaten the same day that it is offered; he shall not leave any of it until the morning."],
['Leviticus','Lev','07','016','16',"But if the sacrifice of his offering \emph{be} a vow, or a voluntary offering, it shall be eaten the same day that he offereth his sacrifice: and on the morrow also the remainder of it shall be eaten:"],
['Leviticus','Lev','07','017','17',"But the remainder of the flesh of the sacrifice on the third day shall be burnt with fire."],
['Leviticus','Lev','07','018','18',"And if \emph{any} of the flesh of the sacrifice of his peace offerings be eaten at all on the third day, it shall not be accepted, neither shall it be imputed unto him that offereth it: it shall be an abomination, and the soul that eateth of it shall bear his iniquity."],
['Leviticus','Lev','07','019','19',"And the flesh that toucheth any unclean \emph{thing} shall not be eaten; it shall be burnt with fire: and as for the flesh, all that be clean shall eat thereof."],
['Leviticus','Lev','07','020','20',"But the soul that eateth \emph{of} the flesh of the sacrifice of peace offerings, that \emph{pertain} unto the LORD, having his uncleanness upon him, even that soul shall be cut off from his people."],
['Leviticus','Lev','07','021','21',"Moreover the soul that shall touch any unclean \emph{thing,} \emph{as} the uncleanness of man, or \emph{any} unclean beast, or any abominable unclean \emph{thing}, and eat of the flesh of the sacrifice of peace offerings, which \emph{pertain} unto the LORD, even that soul shall be cut off from his people."],
['Leviticus','Lev','07','022','22',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','07','023','23',"Speak unto the children of Israel, saying, Ye shall eat no manner of fat, of ox, or of sheep, or of goat."],
['Leviticus','Lev','07','024','24',"And the fat of the beast that dieth of itself, and the fat of that which is torn with beasts, may be used in any other use: but ye shall in no wise eat of it."],
['Leviticus','Lev','07','025','25',"For whosoever eateth the fat of the beast, of which men offer an offering made by fire unto the LORD, even the soul that eateth \emph{it} shall be cut off from his people."],
['Leviticus','Lev','07','026','26',"Moreover ye shall eat no manner of blood, \emph{whether} \emph{it} \emph{be} of fowl or of beast, in any of your dwellings."],
['Leviticus','Lev','07','027','27',"Whatsoever soul \emph{it} \emph{be} that eateth any manner of blood, even that soul shall be cut off from his people."],
['Leviticus','Lev','07','028','28',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','07','029','29',"Speak unto the children of Israel, saying, He that offereth the sacrifice of his peace offerings unto the LORD shall bring his oblation unto the LORD of the sacrifice of his peace offerings."],
['Leviticus','Lev','07','030','30',"His own hands shall bring the offerings of the LORD made by fire, the fat with the breast, it shall he bring, that the breast may be waved \emph{for} a wave offering before the LORD."],
['Leviticus','Lev','07','031','31',"And the priest shall burn the fat upon the altar: but the breast shall be Aaron's and his sons'."],
['Leviticus','Lev','07','032','32',"And the right shoulder shall ye give unto the priest \emph{for} an heave offering of the sacrifices of your peace offerings."],
['Leviticus','Lev','07','033','33',"He among the sons of Aaron, that offereth the blood of the peace offerings, and the fat, shall have the right shoulder for \emph{his} part."],
['Leviticus','Lev','07','034','34',"For the wave breast and the heave shoulder have I taken of the children of Israel from off the sacrifices of their peace offerings, and have given them unto Aaron the priest and unto his sons by a statute for ever from among the children of Israel."],
['Leviticus','Lev','07','035','35',"This \emph{is} \emph{the} \emph{portion} of the anointing of Aaron, and of the anointing of his sons, out of the offerings of the LORD made by fire, in the day \emph{when} he presented them to minister unto the LORD in the priest's office;"],
['Leviticus','Lev','07','036','36',"Which the LORD commanded to be given them of the children of Israel, in the day that he anointed them, \emph{by} a statute for ever throughout their generations."],
['Leviticus','Lev','07','037','37',"This \emph{is} the law of the burnt offering, of the meat offering, and of the sin offering, and of the trespass offering, and of the consecrations, and of the sacrifice of the peace offerings;"],
['Leviticus','Lev','07','038','38',"Which the LORD commanded Moses in mount Sinai, in the day that he commanded the children of Israel to offer their oblations unto the LORD, in the wilderness of Sinai."]
]

Leviticus08 = [
    ['Leviticus','Lev','08','001','1',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','08','002','2',"Take Aaron and his sons with him, and the garments, and the anointing oil, and a bullock for the sin offering, and two rams, and a basket of unleavened bread;"],
['Leviticus','Lev','08','003','3',"And gather thou all the congregation together unto the door of the tabernacle of the congregation."],
['Leviticus','Lev','08','004','4',"And Moses did as the LORD commanded him; and the assembly was gathered together unto the door of the tabernacle of the congregation."],
['Leviticus','Lev','08','005','5',"And Moses said unto the congregation, This \emph{is} the thing which the LORD commanded to be done."],
['Leviticus','Lev','08','006','6',"And Moses brought Aaron and his sons, and washed them with water."],
['Leviticus','Lev','08','007','7',"And he put upon him the coat, and girded him with the girdle, and clothed him with the robe, and put the ephod upon him, and he girded him with the curious girdle of the ephod, and bound \emph{it} unto him therewith."],
['Leviticus','Lev','08','008','8',"And he put the breastplate upon him: also he put in the breastplate the Urim and the Thummim."],
['Leviticus','Lev','08','009','9',"And he put the mitre upon his head; also upon the mitre, \emph{even} upon his forefront, did he put the golden plate, the holy crown; as the LORD commanded Moses."],
['Leviticus','Lev','08','010','10',"And Moses took the anointing oil, and anointed the tabernacle and all that \emph{was} therein, and sanctified them."],
['Leviticus','Lev','08','011','11',"And he sprinkled thereof upon the altar seven times, and anointed the altar and all his vessels, both the laver and his foot, to sanctify them."],
['Leviticus','Lev','08','012','12',"And he poured of the anointing oil upon Aaron's head, and anointed him, to sanctify him."],
['Leviticus','Lev','08','013','13',"And Moses brought Aaron's sons, and put coats upon them, and girded them with girdles, and put bonnets upon them; as the LORD commanded Moses."],
['Leviticus','Lev','08','014','14',"And he brought the bullock for the sin offering: and Aaron and his sons laid their hands upon the head of the bullock for the sin offering."],
['Leviticus','Lev','08','015','15',"And he slew \emph{it}; and Moses took the blood, and put \emph{it} upon the horns of the altar round about with his finger, and purified the altar, and poured the blood at the bottom of the altar, and sanctified it, to make reconciliation upon it."],
['Leviticus','Lev','08','016','16',"And he took all the fat that \emph{was} upon the inwards, and the caul \emph{above} the liver, and the two kidneys, and their fat, and Moses burned \emph{it} upon the altar."],
['Leviticus','Lev','08','017','17',"But the bullock, and his hide, his flesh, and his dung, he burnt with fire without the camp; as the LORD commanded Moses."],
['Leviticus','Lev','08','018','18',"And he brought the ram for the burnt offering: and Aaron and his sons laid their hands upon the head of the ram."],
['Leviticus','Lev','08','019','19',"And he killed \emph{it}; and Moses sprinkled the blood upon the altar round about."],
['Leviticus','Lev','08','020','20',"And he cut the ram into pieces; and Moses burnt the head, and the pieces, and the fat."],
['Leviticus','Lev','08','021','21',"And he washed the inwards and the legs in water; and Moses burnt the whole ram upon the altar: it \emph{was} a burnt sacrifice for a sweet savour, \emph{and} an offering made by fire unto the LORD; as the LORD commanded Moses."],
['Leviticus','Lev','08','022','22',"And he brought the other ram, the ram of consecration: and Aaron and his sons laid their hands upon the head of the ram."],
['Leviticus','Lev','08','023','23',"And he slew \emph{it}; and Moses took of the blood of it, and put \emph{it} upon the tip of Aaron's right ear, and upon the thumb of his right hand, and upon the great toe of his right foot."],
['Leviticus','Lev','08','024','24',"And he brought Aaron's sons, and Moses put of the blood upon the tip of their right ear, and upon the thumbs of their right hands, and upon the great toes of their right feet: and Moses sprinkled the blood upon the altar round about."],
['Leviticus','Lev','08','025','25',"And he took the fat, and the rump, and all the fat that \emph{was} upon the inwards, and the caul \emph{above} the liver, and the two kidneys, and their fat, and the right shoulder:"],
['Leviticus','Lev','08','026','26',"And out of the basket of unleavened bread, that \emph{was} before the LORD, he took one unleavened cake, and a cake of oiled bread, and one wafer, and put \emph{them} on the fat, and upon the right shoulder:"],
['Leviticus','Lev','08','027','27',"And he put all upon Aaron's hands, and upon his sons' hands, and waved them \emph{for} a wave offering before the LORD."],
['Leviticus','Lev','08','028','28',"And Moses took them from off their hands, and burnt \emph{them} on the altar upon the burnt offering: they \emph{were} consecrations for a sweet savour: it \emph{is} an offering made by fire unto the LORD."],
['Leviticus','Lev','08','029','29',"And Moses took the breast, and waved it \emph{for} a wave offering before the LORD: \emph{for} of the ram of consecration it was Moses' part; as the LORD commanded Moses."],
['Leviticus','Lev','08','030','30',"And Moses took of the anointing oil, and of the blood which \emph{was} upon the altar, and sprinkled \emph{it} upon Aaron, \emph{and} upon his garments, and upon his sons, and upon his sons' garments with him; and sanctified Aaron, \emph{and} his garments, and his sons, and his sons' garments with him."],
['Leviticus','Lev','08','031','31',"And Moses said unto Aaron and to his sons, Boil the flesh \emph{at} the door of the tabernacle of the congregation: and there eat it with the bread that \emph{is} in the basket of consecrations, as I commanded, saying, Aaron and his sons shall eat it."],
['Leviticus','Lev','08','032','32',"And that which remaineth of the flesh and of the bread shall ye burn with fire."],
['Leviticus','Lev','08','033','33',"And ye shall not go out of the door of the tabernacle of the congregation \emph{in} seven days, until the days of your consecration be at an end: for seven days shall he consecrate you."],
['Leviticus','Lev','08','034','34',"As he hath done this day, \emph{so} the LORD hath commanded to do, to make an atonement for you."],
['Leviticus','Lev','08','035','35',"Therefore shall ye abide \emph{at} the door of the tabernacle of the congregation day and night seven days, and keep the charge of the LORD, that ye die not: for so I am commanded."],
['Leviticus','Lev','08','036','36',"So Aaron and his sons did all things which the LORD commanded by the hand of Moses."]
]

Leviticus09 = [
    ['Leviticus','Lev','09','001','1',"And it came to pass on the eighth day, \emph{that} Moses called Aaron and his sons, and the elders of Israel;"],
['Leviticus','Lev','09','002','2',"And he said unto Aaron, Take thee a young calf for a sin offering, and a ram for a burnt offering, without blemish, and offer \emph{them} before the LORD."],
['Leviticus','Lev','09','003','3',"And unto the children of Israel thou shalt speak, saying, Take ye a kid of the goats for a sin offering; and a calf and a lamb, \emph{both} of the first year, without blemish, for a burnt offering;"],
['Leviticus','Lev','09','004','4',"Also a bullock and a ram for peace offerings, to sacrifice before the LORD; and a meat offering mingled with oil: for to day the LORD will appear unto you."],
['Leviticus','Lev','09','005','5',"And they brought \emph{that} which Moses commanded before the tabernacle of the congregation: and all the congregation drew near and stood before the LORD."],
['Leviticus','Lev','09','006','6',"And Moses said, This \emph{is} the thing which the LORD commanded that ye should do: and the glory of the LORD shall appear unto you."],
['Leviticus','Lev','09','007','7',"And Moses said unto Aaron, Go unto the altar, and offer thy sin offering, and thy burnt offering, and make an atonement for thyself, and for the people: and offer the offering of the people, and make an atonement for them; as the LORD commanded."],
['Leviticus','Lev','09','008','8',"Aaron therefore went unto the altar, and slew the calf of the sin offering, which \emph{was} for himself."],
['Leviticus','Lev','09','009','9',"And the sons of Aaron brought the blood unto him: and he dipped his finger in the blood, and put \emph{it} upon the horns of the altar, and poured out the blood at the bottom of the altar:"],
['Leviticus','Lev','09','010','10',"But the fat, and the kidneys, and the caul above the liver of the sin offering, he burnt upon the altar; as the LORD commanded Moses."],
['Leviticus','Lev','09','011','11',"And the flesh and the hide he burnt with fire without the camp."],
['Leviticus','Lev','09','012','12',"And he slew the burnt offering; and Aaron's sons presented unto him the blood, which he sprinkled round about upon the altar."],
['Leviticus','Lev','09','013','13',"And they presented the burnt offering unto him, with the pieces thereof, and the head: and he burnt \emph{them} upon the altar."],
['Leviticus','Lev','09','014','14',"And he did wash the inwards and the legs, and burnt \emph{them} upon the burnt offering on the altar."],
['Leviticus','Lev','09','015','15',"And he brought the people's offering, and took the goat, which \emph{was} the sin offering for the people, and slew it, and offered it for sin, as the first."],
['Leviticus','Lev','09','016','16',"And he brought the burnt offering, and offered it according to the manner."],
['Leviticus','Lev','09','017','17',"And he brought the meat offering, and took an handful thereof, and burnt \emph{it} upon the altar, beside the burnt sacrifice of the morning."],
['Leviticus','Lev','09','018','18',"He slew also the bullock and the ram \emph{for} a sacrifice of peace offerings, which \emph{was} for the people: and Aaron's sons presented unto him the blood, which he sprinkled upon the altar round about,"],
['Leviticus','Lev','09','019','19',"And the fat of the bullock and of the ram, the rump, and that which covereth \emph{the} \emph{inwards}, and the kidneys, and the caul \emph{above} the liver:"],
['Leviticus','Lev','09','020','20',"And they put the fat upon the breasts, and he burnt the fat upon the altar:"],
['Leviticus','Lev','09','021','21',"And the breasts and the right shoulder Aaron waved \emph{for} a wave offering before the LORD; as Moses commanded."],
['Leviticus','Lev','09','022','22',"And Aaron lifted up his hand toward the people, and blessed them, and came down from offering of the sin offering, and the burnt offering, and peace offerings."],
['Leviticus','Lev','09','023','23',"And Moses and Aaron went into the tabernacle of the congregation, and came out, and blessed the people: and the glory of the LORD appeared unto all the people."],
['Leviticus','Lev','09','024','24',"And there came a fire out from before the LORD, and consumed upon the altar the burnt offering and the fat: \emph{which} when all the people saw, they shouted, and fell on their faces."]
]

Leviticus10 = [
    ['Leviticus','Leviticus','10','01','1',"And Nadab and Abihu, the sons of Aaron, took either of them his censer, and put fire therein, and put incense thereon, and offered strange fire before the LORD, which he commanded them not."],
    ['Leviticus','Leviticus','10','02','2',"And there went out fire from the LORD, and devoured them, and they died before the LORD."],
    ['Leviticus','Leviticus','10','03','3',"Then Moses said unto Aaron, This \emph{is} \emph{it} that the LORD spake, saying, I will be sanctified in them that come nigh me, and before all the people I will be glorified. And Aaron held his peace."],
    ['Leviticus','Leviticus','10','04','4',"And Moses called Mishael and Elzaphan, the sons of Uzziel the uncle of Aaron, and said unto them, Come near, carry your brethren from before the sanctuary out of the camp."],
    ['Leviticus','Leviticus','10','05','5',"So they went near, and carried them in their coats out of the camp; as Moses had said."],
    ['Leviticus','Leviticus','10','06','6',"And Moses said unto Aaron, and unto Eleazar and unto Ithamar, his sons, Uncover not your heads, neither rend your clothes; lest ye die, and lest wrath come upon all the people: but let your brethren, the whole house of Israel, bewail the burning which the LORD hath kindled."],
    ['Leviticus','Leviticus','10','07','7',"And ye shall not go out from the door of the tabernacle of the congregation, lest ye die: for the anointing oil of the LORD \emph{is} upon you. And they did according to the word of Moses."],
    ['Leviticus','Leviticus','10','08','8',"And the LORD spake unto Aaron, saying,"],
    ['Leviticus','Leviticus','10','09','9',"Do not drink wine nor strong drink, thou, nor thy sons with thee, when ye go into the tabernacle of the congregation, lest ye die: \emph{it} \emph{shall} \emph{be} a statute for ever throughout your generations:"],
    ['Leviticus','Leviticus','10','10','10',"And that ye may put difference between holy and unholy, and between unclean and clean;"],
    ['Leviticus','Leviticus','10','11','11',"And that ye may teach the children of Israel all the statutes which the LORD hath spoken unto them by the hand of Moses."],
    ['Leviticus','Leviticus','10','12','12',"And Moses spake unto Aaron, and unto Eleazar and unto Ithamar, his sons that were left, Take the meat offering that remaineth of the offerings of the LORD made by fire, and eat it without leaven beside the altar: for it \emph{is} most holy:"],
    ['Leviticus','Leviticus','10','13','13',"And ye shall eat it in the holy place, because it \emph{is} thy due, and thy sons' due, of the sacrifices of the LORD made by fire: for so I am commanded."],
    ['Leviticus','Leviticus','10','14','14',"And the wave breast and heave shoulder shall ye eat in a clean place; thou, and thy sons, and thy daughters with thee: for \emph{they} \emph{be} thy due, and thy sons' due, \emph{which} are given out of the sacrifices of peace offerings of the children of Israel."],
    ['Leviticus','Leviticus','10','15','15',"The heave shoulder and the wave breast shall they bring with the offerings made by fire of the fat, to wave \emph{it} \emph{for} a wave offering before the LORD; and it shall be thine, and thy sons' with thee, by a statute for ever; as the LORD hath commanded."],
    ['Leviticus','Leviticus','10','16','16',"And Moses diligently sought the goat of the sin offering, and, behold, it was burnt: and he was angry with Eleazar and Ithamar, the sons of Aaron \emph{which} \emph{were} left \emph{alive}, saying,"],
    ['Leviticus','Leviticus','10','17','17',"Wherefore have ye not eaten the sin offering in the holy place, seeing it \emph{is} most holy, and \emph{God} hath given it you to bear the iniquity of the congregation, to make atonement for them before the LORD?"],
    ['Leviticus','Leviticus','10','18','18',"Behold, the blood of it was not brought in within the holy \emph{place}: ye should indeed have eaten it in the holy \emph{place}, as I commanded."],
    ['Leviticus','Leviticus','10','19','19',"And Aaron said unto Moses, Behold, this day have they offered their sin offering and their burnt offering before the LORD; and such things have befallen me: and \emph{if} I had eaten the sin offering to day, should it have been accepted in the sight of the LORD?"],
    ['Leviticus','Leviticus','10','20','20',"And when Moses heard \emph{that}, he was content."]]

Leviticus11 = [
    ['Leviticus','Leviticus','11','01','1',"And the LORD spake unto Moses and to Aaron, saying unto them,"],
    ['Leviticus','Leviticus','11','02','2',"Speak unto the children of Israel, saying, These \emph{are} the beasts which ye shall eat among all the beasts that \emph{are} on the earth."],
    ['Leviticus','Leviticus','11','03','3',"Whatsoever parteth the hoof, and is clovenfooted, \emph{and} cheweth the cud, among the beasts, that shall ye eat."],
    ['Leviticus','Leviticus','11','04','4',"Nevertheless these shall ye not eat of them that chew the cud, or of them that divide the hoof: \emph{as} the camel, because he cheweth the cud, but divideth not the hoof; he \emph{is} unclean unto you."],
    ['Leviticus','Leviticus','11','05','5',"And the coney, because he cheweth the cud, but divideth not the hoof; he \emph{is} unclean unto you."],
    ['Leviticus','Leviticus','11','06','6',"And the hare, because he cheweth the cud, but divideth not the hoof; he \emph{is} unclean unto you."],
    ['Leviticus','Leviticus','11','07','7',"And the swine, though he divide the hoof, and be clovenfooted, yet he cheweth not the cud; he \emph{is} unclean to you."],
    ['Leviticus','Leviticus','11','08','8',"Of their flesh shall ye not eat, and their carcase shall ye not touch; they \emph{are} unclean to you."],
    ['Leviticus','Leviticus','11','09','9',"These shall ye eat of all that \emph{are} in the waters: whatsoever hath fins and scales in the waters, in the seas, and in the rivers, them shall ye eat."],
    ['Leviticus','Leviticus','11','10','10',"And all that have not fins and scales in the seas, and in the rivers, of all that move in the waters, and of any living thing which \emph{is} in the waters, they \emph{shall} \emph{be} an abomination unto you:"],
    ['Leviticus','Leviticus','11','11','11',"They shall be even an abomination unto you; ye shall not eat of their flesh, but ye shall have their carcases in abomination."],
    ['Leviticus','Leviticus','11','12','12',"Whatsoever hath no fins nor scales in the waters, that \emph{shall} \emph{be} an abomination unto you."],
    ['Leviticus','Leviticus','11','13','13',"And these \emph{are} \emph{they} \emph{which} ye shall have in abomination among the fowls; they shall not be eaten, they \emph{are} an abomination: the eagle, and the ossifrage, and the ospray,"],
    ['Leviticus','Leviticus','11','14','14',"And the vulture, and the kite after his kind;"],
    ['Leviticus','Leviticus','11','15','15',"Every raven after his kind;"],
    ['Leviticus','Leviticus','11','16','16',"And the owl, and the night hawk, and the cuckow, and the hawk after his kind,"],
    ['Leviticus','Leviticus','11','17','17',"And the little owl, and the cormorant, and the great owl,"],
    ['Leviticus','Leviticus','11','18','18',"And the swan, and the pelican, and the gier eagle,"],
    ['Leviticus','Leviticus','11','19','19',"And the stork, the heron after her kind, and the lapwing, and the bat."],
    ['Leviticus','Leviticus','11','20','20',"All fowls that creep, going upon \emph{all} four, \emph{shall} \emph{be} an abomination unto you."],
    ['Leviticus','Leviticus','11','21','21',"Yet these may ye eat of every flying creeping thing that goeth upon \emph{all} four, which have legs above their feet, to leap withal upon the earth;"],
    ['Leviticus','Leviticus','11','22','22',"\emph{Even} these of them ye may eat; the locust after his kind, and the bald locust after his kind, and the beetle after his kind, and the grasshopper after his kind."],
    ['Leviticus','Leviticus','11','23','23',"But all \emph{other} flying creeping things, which have four feet, \emph{shall} \emph{be} an abomination unto you."],
    ['Leviticus','Leviticus','11','24','24',"And for these ye shall be unclean: whosoever toucheth the carcase of them shall be unclean until the even."],
    ['Leviticus','Leviticus','11','25','25',"And whosoever beareth \emph{ought} of the carcase of them shall wash his clothes, and be unclean until the even."],
    ['Leviticus','Leviticus','11','26','26',"\emph{The} \emph{carcases} of every beast which divideth the hoof, and \emph{is} not clovenfooted, nor cheweth the cud, \emph{are} unclean unto you: every one that toucheth them shall be unclean."],
    ['Leviticus','Leviticus','11','27','27',"And whatsoever goeth upon his paws, among all manner of beasts that go on \emph{all} four, those \emph{are} unclean unto you: whoso toucheth their carcase shall be unclean until the even."],
    ['Leviticus','Leviticus','11','28','28',"And he that beareth the carcase of them shall wash his clothes, and be unclean until the even: they \emph{are} unclean unto you."],
    ['Leviticus','Leviticus','11','29','29',"These also \emph{shall} \emph{be} unclean unto you among the creeping things that creep upon the earth; the weasel, and the mouse, and the tortoise after his kind,"],
    ['Leviticus','Leviticus','11','30','30',"And the ferret, and the chameleon, and the lizard, and the snail, and the mole."],
    ['Leviticus','Leviticus','11','31','31',"These \emph{are} unclean to you among all that creep: whosoever doth touch them, when they be dead, shall be unclean until the even."],
    ['Leviticus','Leviticus','11','32','32',"And upon whatsoever \emph{any} of them, when they are dead, doth fall, it shall be unclean; whether \emph{it} \emph{be} any vessel of wood, or raiment, or skin, or sack, whatsoever vessel \emph{it} \emph{be}, wherein \emph{any} work is done, it must be put into water, and it shall be unclean until the even; so it shall be cleansed."],
    ['Leviticus','Leviticus','09','33','33',"And every earthen vessel, whereinto \emph{any} of them falleth, whatsoever \emph{is} in it shall be unclean; and ye shall break it."],
    ['Leviticus','Leviticus','11','34','34',"Of all meat which may be eaten, \emph{that} on which \emph{such} water cometh shall be unclean: and all drink that may be drunk in every \emph{such} vessel shall be unclean."],
    ['Leviticus','Leviticus','11','35','35',"And every \emph{thing} whereupon \emph{any} \emph{part} of their carcase falleth shall be unclean; \emph{whether} \emph{it} \emph{be} oven, or ranges for pots, they shall be broken down: \emph{for} they \emph{are} unclean, and shall be unclean unto you."],
    ['Leviticus','Leviticus','11','36','36',"Nevertheless a fountain or pit, \emph{wherein} \emph{there} \emph{is} plenty of water, shall be clean: but that which toucheth their carcase shall be unclean."],
    ['Leviticus','Leviticus','11','37','37',"And if \emph{any} \emph{part} of their carcase fall upon any sowing seed which is to be sown, it \emph{shall} \emph{be} clean."],
    ['Leviticus','Leviticus','11','38','38',"But if \emph{any} water be put upon the seed, and \emph{any} \emph{part} of their carcase fall thereon, it \emph{shall} \emph{be} unclean unto you."],
    ['Leviticus','Leviticus','11','39','39',"And if any beast, of which ye may eat, die; he that toucheth the carcase thereof shall be unclean until the even."],
    ['Leviticus','Leviticus','11','40','40',"And he that eateth of the carcase of it shall wash his clothes, and be unclean until the even: he also that beareth the carcase of it shall wash his clothes, and be unclean until the even."],
    ['Leviticus','Leviticus','11','41','41',"And every creeping thing that creepeth upon the earth \emph{shall} \emph{be} an abomination; it shall not be eaten."],
    ['Leviticus','Leviticus','11','42','42',"Whatsoever goeth upon the belly, and whatsoever goeth upon \emph{all} four, or whatsoever hath more feet among all creeping things that creep upon the earth, them ye shall not eat; for they \emph{are} an abomination."],
    ['Leviticus','Leviticus','11','43','43',"Ye shall not make yourselves abominable with any creeping thing that creepeth, neither shall ye make yourselves unclean with them, that ye should be defiled thereby."],
    ['Leviticus','Leviticus','11','44','44',"For I \emph{am} the LORD your God: ye shall therefore sanctify yourselves, and ye shall be holy; for I \emph{am} holy: neither shall ye defile yourselves with any manner of creeping thing that creepeth upon the earth."],
    ['Leviticus','Leviticus','11','45','45',"For I \emph{am} the LORD that bringeth you up out of the land of Egypt, to be your God: ye shall therefore be holy, for I \emph{am} holy."],
    ['Leviticus','Leviticus','11','46','46',"This \emph{is} the law of the beasts, and of the fowl, and of every living creature that moveth in the waters, and of every creature that creepeth upon the earth:"],
    ['Leviticus','Leviticus','11','47','47',"To make a difference between the unclean and the clean, and between the beast that may be eaten and the beast that may not be eaten."]]

Leviticus12 = [
    ['Leviticus','Leviticus','12','01','1',"And the LORD spake unto Moses, saying,"],
    ['Leviticus','Leviticus','12','02','2',"Speak unto the children of Israel, saying, If a woman have conceived seed, and born a man child: then she shall be unclean seven days; according to the days of the separation for her infirmity shall she be unclean."],
    ['Leviticus','Leviticus','12','03','3',"And in the eighth day the flesh of his foreskin shall be circumcised."],
    ['Leviticus','Leviticus','12','04','4',"And she shall then continue in the blood of her purifying three and thirty days; she shall touch no hallowed thing, nor come into the sanctuary, until the days of her purifying be fulfilled."],
    ['Leviticus','Leviticus','12','05','5',"But if she bear a maid child, then she shall be unclean two weeks, as in her separation: and she shall continue in the blood of her purifying threescore and six days."],
    ['Leviticus','Leviticus','12','06','6',"And when the days of her purifying are fulfilled, for a son, or for a daughter, she shall bring a lamb of the first year for a burnt offering, and a young pigeon, or a turtledove, for a sin offering, unto the door of the tabernacle of the congregation, unto the priest:"],
    ['Leviticus','Leviticus','12','07','7',"Who shall offer it before the LORD, and make an atonement for her; and she shall be cleansed from the issue of her blood. This \emph{is} the law for her that hath born a male or a female."],
    ['Leviticus','Leviticus','12','08','8',"And if she be not able to bring a lamb, then she shall bring two turtles, or two young pigeons; the one for the burnt offering, and the other for a sin offering: and the priest shall make an atonement for her, and she shall be clean."]]

Leviticus13 = [
    ['Leviticus','Leviticus','13','01','1',"And the LORD spake unto Moses and Aaron, saying,"],
    ['Leviticus','Leviticus','13','02','2',"When a man shall have in the skin of his flesh a rising, a scab, or bright spot, and it be in the skin of his flesh \emph{like} the plague of leprosy; then he shall be brought unto Aaron the priest, or unto one of his sons the priests:"],
    ['Leviticus','Leviticus','13','03','3',"And the priest shall look on the plague in the skin of the flesh: and \emph{when} the hair in the plague is turned white, and the plague in sight \emph{be} deeper than the skin of his flesh, it \emph{is} a plague of leprosy: and the priest shall look on him, and pronounce him unclean."],
    ['Leviticus','Leviticus','13','04','4',"If the bright spot \emph{be} white in the skin of his flesh, and in sight \emph{be} not deeper than the skin, and the hair thereof be not turned white; then the priest shall shut up \emph{him} \emph{that} \emph{hath} the plague seven days:"],
    ['Leviticus','Leviticus','13','05','5',"And the priest shall look on him the seventh day: and, behold, \emph{if} the plague in his sight be at a stay, \emph{and} the plague spread not in the skin; then the priest shall shut him up seven days more:"],
    ['Leviticus','Leviticus','13','06','6',"And the priest shall look on him again the seventh day: and, behold, \emph{if} the plague \emph{be} somewhat dark, \emph{and} the plague spread not in the skin, the priest shall pronounce him clean: it \emph{is} \emph{but} a scab: and he shall wash his clothes, and be clean."],
    ['Leviticus','Leviticus','13','07','7',"But if the scab spread much abroad in the skin, after that he hath been seen of the priest for his cleansing, he shall be seen of the priest again:"],
    ['Leviticus','Leviticus','13','08','8',"And \emph{if} the priest see that, behold, the scab spreadeth in the skin, then the priest shall pronounce him unclean: it \emph{is} a leprosy."],
    ['Leviticus','Leviticus','13','09','9',"When the plague of leprosy is in a man, then he shall be brought unto the priest;"],
    ['Leviticus','Leviticus','13','10','10',"And the priest shall see \emph{him}: and, behold, \emph{if} the rising \emph{be} white in the skin, and it have turned the hair white, and \emph{there} \emph{be} quick raw flesh in the rising;"],
    ['Leviticus','Leviticus','13','11','11',"It \emph{is} an old leprosy in the skin of his flesh, and the priest shall pronounce him unclean, and shall not shut him up: for he \emph{is} unclean."],
    ['Leviticus','Leviticus','13','12','12',"And if a leprosy break out abroad in the skin, and the leprosy cover all the skin of \emph{him} \emph{that} \emph{hath} the plague from his head even to his foot, wheresoever the priest looketh;"],
    ['Leviticus','Leviticus','13','13','13',"Then the priest shall consider: and, behold, \emph{if} the leprosy have covered all his flesh, he shall pronounce \emph{him} clean \emph{that} \emph{hath} the plague: it is all turned white: he \emph{is} clean."],
    ['Leviticus','Leviticus','13','14','14',"But when raw flesh appeareth in him, he shall be unclean."],
    ['Leviticus','Leviticus','13','15','15',"And the priest shall see the raw flesh, and pronounce him to be unclean: \emph{for} the raw flesh \emph{is} unclean: it \emph{is} a leprosy."],
    ['Leviticus','Leviticus','13','16','16',"Or if the raw flesh turn again, and be changed unto white, he shall come unto the priest;"],
    ['Leviticus','Leviticus','13','17','17',"And the priest shall see him: and, behold, \emph{if} the plague be turned into white; then the priest shall pronounce \emph{him} clean \emph{that} \emph{hath} the plague: he \emph{is} clean."],
    ['Leviticus','Leviticus','13','18','18',"The flesh also, in which, \emph{even} in the skin thereof, was a boil, and is healed,"],
    ['Leviticus','Leviticus','13','19','19',"And in the place of the boil there be a white rising, or a bright spot, white, and somewhat reddish, and it be shewed to the priest;"],
    ['Leviticus','Leviticus','13','20','20',"And if, when the priest seeth it, behold, it \emph{be} in sight lower than the skin, and the hair thereof be turned white; the priest shall pronounce him unclean: it \emph{is} a plague of leprosy broken out of the boil."],
    ['Leviticus','Leviticus','13','21','21',"But if the priest look on it, and, behold, \emph{there} \emph{be} no white hairs therein, and \emph{if} it \emph{be} not lower than the skin, but \emph{be} somewhat dark; then the priest shall shut him up seven days:"],
    ['Leviticus','Leviticus','13','22','22',"And if it spread much abroad in the skin, then the priest shall pronounce him unclean: it \emph{is} a plague."],
    ['Leviticus','Leviticus','13','23','23',"But if the bright spot stay in his place, \emph{and} spread not, it \emph{is} a burning boil; and the priest shall pronounce him clean."],
    ['Leviticus','Leviticus','13','24','24',"Or if there be \emph{any} flesh, in the skin whereof \emph{there} \emph{is} a hot burning, and the quick \emph{flesh} that burneth have a white bright spot, somewhat reddish, or white;"],
    ['Leviticus','Leviticus','13','25','25',"Then the priest shall look upon it: and, behold, \emph{if} the hair in the bright spot be turned white, and it \emph{be} \emph{in} sight deeper than the skin; it \emph{is} a leprosy broken out of the burning: wherefore the priest shall pronounce him unclean: it \emph{is} the plague of leprosy."],
    ['Leviticus','Leviticus','13','26','26',"But if the priest look on it, and, behold, \emph{there} \emph{be} no white hair in the bright spot, and it \emph{be} no lower than the \emph{other} skin, but \emph{be} somewhat dark; then the priest shall shut him up seven days:"],
    ['Leviticus','Leviticus','13','27','27',"And the priest shall look upon him the seventh day: \emph{and} if it be spread much abroad in the skin, then the priest shall pronounce him unclean: it \emph{is} the plague of leprosy."],
    ['Leviticus','Leviticus','13','28','28',"And if the bright spot stay in his place, \emph{and} spread not in the skin, but it \emph{be} somewhat dark; it \emph{is} a rising of the burning, and the priest shall pronounce him clean: for it \emph{is} an inflammation of the burning."],
    ['Leviticus','Leviticus','13','29','29',"If a man or woman have a plague upon the head or the beard;"],
    ['Leviticus','Leviticus','13','30','30',"Then the priest shall see the plague: and, behold, if it \emph{be} in sight deeper than the skin; \emph{and} \emph{there} \emph{be} in it a yellow thin hair; then the priest shall pronounce him unclean: it \emph{is} a dry scall, \emph{even} a leprosy upon the head or beard."],
    ['Leviticus','Leviticus','13','31','31',"And if the priest look on the plague of the scall, and, behold, it \emph{be} not in sight deeper than the skin, and \emph{that} \emph{there} \emph{is} no black hair in it; then the priest shall shut up \emph{him} \emph{that} \emph{hath} the plague of the scall seven days:"],
    ['Leviticus','Leviticus','13','32','32',"And in the seventh day the priest shall look on the plague: and, behold, \emph{if} the scall spread not, and there be in it no yellow hair, and the scall \emph{be} not in sight deeper than the skin;"],
    ['Leviticus','Leviticus','13','33','33',"He shall be shaven, but the scall shall he not shave; and the priest shall shut up \emph{him} \emph{that} \emph{hath} the scall seven days more:"],
    ['Leviticus','Leviticus','13','34','34',"And in the seventh day the priest shall look on the scall: and, behold, \emph{if} the scall be not spread in the skin, nor \emph{be} in sight deeper than the skin; then the priest shall pronounce him clean: and he shall wash his clothes, and be clean."],
    ['Leviticus','Leviticus','13','35','35',"But if the scall spread much in the skin after his cleansing;"],
    ['Leviticus','Leviticus','13','36','36',"Then the priest shall look on him: and, behold, if the scall be spread in the skin, the priest shall not seek for yellow hair; he \emph{is} unclean."],
    ['Leviticus','Leviticus','13','37','37',"But if the scall be in his sight at a stay, and \emph{that} there is black hair grown up therein; the scall is healed, he \emph{is} clean: and the priest shall pronounce him clean."],
    ['Leviticus','Leviticus','13','38','38',"If a man also or a woman have in the skin of their flesh bright spots, \emph{even} white bright spots;"],
    ['Leviticus','Leviticus','13','39','39',"Then the priest shall look: and, behold, \emph{if} the bright spots in the skin of their flesh \emph{be} darkish white; it \emph{is} a freckled spot \emph{that} groweth in the skin; he \emph{is} clean."],
    ['Leviticus','Leviticus','13','40','40',"And the man whose hair is fallen off his head, he \emph{is} bald; \emph{yet} \emph{is} he clean."],
    ['Leviticus','Leviticus','13','41','41',"And he that hath his hair fallen off from the part of his head toward his face, he \emph{is} forehead bald: \emph{yet} \emph{is} he clean."],
    ['Leviticus','Leviticus','13','42','42',"And if there be in the bald head, or bald forehead, a white reddish sore; it \emph{is} a leprosy sprung up in his bald head, or his bald forehead."],
    ['Leviticus','Leviticus','13','43','43',"Then the priest shall look upon it: and, behold, \emph{if} the rising of the sore \emph{be} white reddish in his bald head, or in his bald forehead, as the leprosy appeareth in the skin of the flesh;"],
    ['Leviticus','Leviticus','13','44','44',"He is a leprous man, he \emph{is} unclean: the priest shall pronounce him utterly unclean; his plague \emph{is} in his head."],
    ['Leviticus','Leviticus','13','45','45',"And the leper in whom the plague \emph{is}, his clothes shall be rent, and his head bare, and he shall put a covering upon his upper lip, and shall cry, Unclean, unclean."],
    ['Leviticus','Leviticus','13','46','46',"All the days wherein the plague \emph{shall} \emph{be} in him he shall be defiled; he \emph{is} unclean: he shall dwell alone; without the camp \emph{shall} his habitation \emph{be}."],
    ['Leviticus','Leviticus','13','47','47',"The garment also that the plague of leprosy is in, \emph{whether} \emph{it} \emph{be} a woollen garment, or a linen garment;"],
    ['Leviticus','Leviticus','13','48','48',"Whether \emph{it} \emph{be} in the warp, or woof; of linen, or of woollen; whether in a skin, or in any thing made of skin;"],
    ['Leviticus','Leviticus','13','49','49',"And if the plague be greenish or reddish in the garment, or in the skin, either in the warp, or in the woof, or in any thing of skin; it \emph{is} a plague of leprosy, and shall be shewed unto the priest:"],
    ['Leviticus','Leviticus','13','50','50',"And the priest shall look upon the plague, and shut up \emph{it} \emph{that} \emph{hath} the plague seven days:"],
    ['Leviticus','Leviticus','13','51','51',"And he shall look on the plague on the seventh day: if the plague be spread in the garment, either in the warp, or in the woof, or in a skin, \emph{or} in any work that is made of skin; the plague \emph{is} a fretting leprosy; it \emph{is} unclean."],
    ['Leviticus','Leviticus','13','52','52',"He shall therefore burn that garment, whether warp or woof, in woollen or in linen, or any thing of skin, wherein the plague is: for it \emph{is} a fretting leprosy; it shall be burnt in the fire."],
    ['Leviticus','Leviticus','13','53','53',"And if the priest shall look, and, behold, the plague be not spread in the garment, either in the warp, or in the woof, or in any thing of skin;"],
    ['Leviticus','Leviticus','13','54','54',"Then the priest shall command that they wash \emph{the} \emph{thing} wherein the plague \emph{is}, and he shall shut it up seven days more:"],
    ['Leviticus','Leviticus','13','55','55',"And the priest shall look on the plague, after that it is washed: and, behold, \emph{if} the plague have not changed his colour, and the plague be not spread; it \emph{is} unclean; thou shalt burn it in the fire; it \emph{is} fret inward, \emph{whether} it \emph{be} bare within or without."],
    ['Leviticus','Leviticus','13','56','56',"And if the priest look, and, behold, the plague \emph{be} somewhat dark after the washing of it; then he shall rend it out of the garment, or out of the skin, or out of the warp, or out of the woof:"],
    ['Leviticus','Leviticus','13','57','57',"And if it appear still in the garment, either in the warp, or in the woof, or in any thing of skin; it \emph{is} a spreading \emph{plague}: thou shalt burn that wherein the plague \emph{is} with fire."],
    ['Leviticus','Leviticus','13','58','58',"And the garment, either warp, or woof, or whatsoever thing of skin \emph{it} \emph{be}, which thou shalt wash, if the plague be departed from them, then it shall be washed the second time, and shall be clean."],
    ['Leviticus','Leviticus','13','59','59',"This \emph{is} the law of the plague of leprosy in a garment of woollen or linen, either in the warp, or woof, or any thing of skins, to pronounce it clean, or to pronounce it unclean."]
]

Leviticus14 = [
    ['Leviticus','Leviticus','14','01','1',"And the LORD spake unto Moses, saying,"],
    ['Leviticus','Leviticus','14','02','2',"This shall be the law of the leper in the day of his cleansing: He shall be brought unto the priest:"],
    ['Leviticus','Leviticus','14','03','3',"And the priest shall go forth out of the camp; and the priest shall look, and, behold, \emph{if} the plague of leprosy be healed in the leper;"],
    ['Leviticus','Leviticus','14','04','4',"Then shall the priest command to take for him that is to be cleansed two birds alive \emph{and} clean, and cedar wood, and scarlet, and hyssop:"],
    ['Leviticus','Leviticus','14','05','5',"And the priest shall command that one of the birds be killed in an earthen vessel over running water:"],
    ['Leviticus','Leviticus','14','06','6',"As for the living bird, he shall take it, and the cedar wood, and the scarlet, and the hyssop, and shall dip them and the living bird in the blood of the bird \emph{that} \emph{was} killed over the running water:"],
    ['Leviticus','Leviticus','14','07','7',"And he shall sprinkle upon him that is to be cleansed from the leprosy seven times, and shall pronounce him clean, and shall let the living bird loose into the open field."],
    ['Leviticus','Leviticus','14','08','8',"And he that is to be cleansed shall wash his clothes, and shave off all his hair, and wash himself in water, that he may be clean: and after that he shall come into the camp, and shall tarry abroad out of his tent seven days."],
    ['Leviticus','Leviticus','14','09','9',"But it shall be on the seventh day, that he shall shave all his hair off his head and his beard and his eyebrows, even all his hair he shall shave off: and he shall wash his clothes, also he shall wash his flesh in water, and he shall be clean."],
    ['Leviticus','Leviticus','14','10','10',"And on the eighth day he shall take two he lambs without blemish, and one ewe lamb of the first year without blemish, and three tenth deals of fine flour \emph{for} a meat offering, mingled with oil, and one log of oil."],
    ['Leviticus','Leviticus','14','11','11',"And the priest that maketh \emph{him} clean shall present the man that is to be made clean, and those things, before the LORD, \emph{at} the door of the tabernacle of the congregation:"],
    ['Leviticus','Leviticus','14','12','12',"And the priest shall take one he lamb, and offer him for a trespass offering, and the log of oil, and wave them \emph{for} a wave offering before the LORD:"],
    ['Leviticus','Leviticus','14','13','13',"And he shall slay the lamb in the place where he shall kill the sin offering and the burnt offering, in the holy place: for as the sin offering \emph{is} the priest's, \emph{so} \emph{is} the trespass offering: it \emph{is} most holy:"],
    ['Leviticus','Leviticus','14','14','14',"And the priest shall take \emph{some} of the blood of the trespass offering, and the priest shall put \emph{it} upon the tip of the right ear of him that is to be cleansed, and upon the thumb of his right hand, and upon the great toe of his right foot:"],
    ['Leviticus','Leviticus','14','15','15',"And the priest shall take \emph{some} of the log of oil, and pour \emph{it} into the palm of his own left hand:"],
    ['Leviticus','Leviticus','14','16','16',"And the priest shall dip his right finger in the oil that \emph{is} in his left hand, and shall sprinkle of the oil with his finger seven times before the LORD:"],
    ['Leviticus','Leviticus','14','17','17',"And of the rest of the oil that \emph{is} in his hand shall the priest put upon the tip of the right ear of him that is to be cleansed, and upon the thumb of his right hand, and upon the great toe of his right foot, upon the blood of the trespass offering:"],
    ['Leviticus','Leviticus','14','18','18',"And the remnant of the oil that \emph{is} in the priest's hand he shall pour upon the head of him that is to be cleansed: and the priest shall make an atonement for him before the LORD."],
    ['Leviticus','Leviticus','14','19','19',"And the priest shall offer the sin offering, and make an atonement for him that is to be cleansed from his uncleanness; and afterward he shall kill the burnt offering:"],
    ['Leviticus','Leviticus','14','20','20',"And the priest shall offer the burnt offering and the meat offering upon the altar: and the priest shall make an atonement for him, and he shall be clean."],
    ['Leviticus','Leviticus','14','21','21',"And if he \emph{be} poor, and cannot get so much; then he shall take one lamb \emph{for} a trespass offering to be waved, to make an atonement for him, and one tenth deal of fine flour mingled with oil for a meat offering, and a log of oil;"],
    ['Leviticus','Leviticus','14','22','22',"And two turtledoves, or two young pigeons, such as he is able to get; and the one shall be a sin offering, and the other a burnt offering."],
    ['Leviticus','Leviticus','14','23','23',"And he shall bring them on the eighth day for his cleansing unto the priest, unto the door of the tabernacle of the congregation, before the LORD."],
    ['Leviticus','Leviticus','14','24','24',"And the priest shall take the lamb of the trespass offering, and the log of oil, and the priest shall wave them \emph{for} a wave offering before the LORD:"],
    ['Leviticus','Leviticus','14','25','25',"And he shall kill the lamb of the trespass offering, and the priest shall take \emph{some} of the blood of the trespass offering, and put \emph{it} upon the tip of the right ear of him that is to be cleansed, and upon the thumb of his right hand, and upon the great toe of his right foot:"],
    ['Leviticus','Leviticus','14','26','26',"And the priest shall pour of the oil into the palm of his own left hand:"],
    ['Leviticus','Leviticus','14','27','27',"And the priest shall sprinkle with his right finger \emph{some} of the oil that \emph{is} in his left hand seven times before the LORD:"],
    ['Leviticus','Leviticus','14','28','28',"And the priest shall put of the oil that \emph{is} in his hand upon the tip of the right ear of him that is to be cleansed, and upon the thumb of his right hand, and upon the great toe of his right foot, upon the place of the blood of the trespass offering:"],
    ['Leviticus','Leviticus','14','29','29',"And the rest of the oil that \emph{is} in the priest's hand he shall put upon the head of him that is to be cleansed, to make an atonement for him before the LORD."],
    ['Leviticus','Leviticus','14','30','30',"And he shall offer the one of the turtledoves, or of the young pigeons, such as he can get;"],
    ['Leviticus','Leviticus','14','31','31',"\emph{Even} such as he is able to get, the one \emph{for} a sin offering, and the other \emph{for} a burnt offering, with the meat offering: and the priest shall make an atonement for him that is to be cleansed before the LORD."],
    ['Leviticus','Leviticus','14','32','32',"This \emph{is} the law \emph{of} \emph{him} in whom \emph{is} the plague of leprosy, whose hand is not able to get \emph{that} \emph{which} \emph{pertaineth} to his cleansing."],
    ['Leviticus','Leviticus','14','33','33',"And the LORD spake unto Moses and unto Aaron, saying,"],
    ['Leviticus','Leviticus','14','34','34',"When ye be come into the land of Canaan, which I give to you for a possession, and I put the plague of leprosy in a house of the land of your possession;"],
    ['Leviticus','Leviticus','14','35','35',"And he that owneth the house shall come and tell the priest, saying, It seemeth to me \emph{there} \emph{is} as it were a plague in the house:"],
    ['Leviticus','Leviticus','14','36','36',"Then the priest shall command that they empty the house, before the priest go \emph{into} \emph{it} to see the plague, that all that \emph{is} in the house be not made unclean: and afterward the priest shall go in to see the house:"],
    ['Leviticus','Leviticus','14','37','37',"And he shall look on the plague, and, behold, \emph{if} the plague \emph{be} in the walls of the house with hollow strakes, greenish or reddish, which in sight \emph{are} lower than the wall;"],
    ['Leviticus','Leviticus','14','38','38',"Then the priest shall go out of the house to the door of the house, and shut up the house seven days:"],
    ['Leviticus','Leviticus','14','39','39',"And the priest shall come again the seventh day, and shall look: and, behold, \emph{if} the plague be spread in the walls of the house;"],
    ['Leviticus','Leviticus','14','40','40',"Then the priest shall command that they take away the stones in which the plague \emph{is}, and they shall cast them into an unclean place without the city:"],
    ['Leviticus','Leviticus','14','41','41',"And he shall cause the house to be scraped within round about, and they shall pour out the dust that they scrape off without the city into an unclean place:"],
    ['Leviticus','Leviticus','14','42','42',"And they shall take other stones, and put \emph{them} in the place of those stones; and he shall take other morter, and shall plaister the house."],
    ['Leviticus','Leviticus','14','43','43',"And if the plague come again, and break out in the house, after that he hath taken away the stones, and after he hath scraped the house, and after it is plaistered;"],
    ['Leviticus','Leviticus','14','44','44',"Then the priest shall come and look, and, behold, \emph{if} the plague be spread in the house, it \emph{is} a fretting leprosy in the house: it \emph{is} unclean."],
    ['Leviticus','Leviticus','14','45','45',"And he shall break down the house, the stones of it, and the timber thereof, and all the morter of the house; and he shall carry \emph{them} forth out of the city into an unclean place."],
    ['Leviticus','Leviticus','14','46','46',"Moreover he that goeth into the house all the while that it is shut up shall be unclean until the even."],
    ['Leviticus','Leviticus','14','47','47',"And he that lieth in the house shall wash his clothes; and he that eateth in the house shall wash his clothes."],
    ['Leviticus','Leviticus','14','48','48',"And if the priest shall come in, and look \emph{upon} \emph{it}, and, behold, the plague hath not spread in the house, after the house was plaistered: then the priest shall pronounce the house clean, because the plague is healed."],
    ['Leviticus','Leviticus','14','49','49',"And he shall take to cleanse the house two birds, and cedar wood, and scarlet, and hyssop:"],
    ['Leviticus','Leviticus','14','50','50',"And he shall kill the one of the birds in an earthen vessel over running water:"],
    ['Leviticus','Leviticus','14','51','51',"And he shall take the cedar wood, and the hyssop, and the scarlet, and the living bird, and dip them in the blood of the slain bird, and in the running water, and sprinkle the house seven times:"],
    ['Leviticus','Leviticus','14','52','52',"And he shall cleanse the house with the blood of the bird, and with the running water, and with the living bird, and with the cedar wood, and with the hyssop, and with the scarlet:"],
    ['Leviticus','Leviticus','14','53','53',"But he shall let go the living bird out of the city into the open fields, and make an atonement for the house: and it shall be clean."],
    ['Leviticus','Leviticus','14','54','54',"This \emph{is} the law for all manner of plague of leprosy, and scall,"],
    ['Leviticus','Leviticus','14','55','55',"And for the leprosy of a garment, and of a house,"],
    ['Leviticus','Leviticus','14','56','56',"And for a rising, and for a scab, and for a bright spot:"],
    ['Leviticus','Leviticus','14','57','57',"To teach when \emph{it} \emph{is} unclean, and when \emph{it} \emph{is} clean: this \emph{is} the law of leprosy."]]

Leviticus15 = [
    ['Leviticus','Leviticus','15','01','1',"And the LORD spake unto Moses and to Aaron, saying,"],
    ['Leviticus','Leviticus','15','02','2',"Speak unto the children of Israel, and say unto them, When any man hath a running issue out of his flesh, \emph{because} \emph{of} his issue he \emph{is} unclean."],
    ['Leviticus','Leviticus','15','03','3',"And this shall be his uncleanness in his issue: whether his flesh run with his issue, or his flesh be stopped from his issue, it \emph{is} his uncleanness."],
    ['Leviticus','Leviticus','15','04','4',"Every bed, whereon he lieth that hath the issue, is unclean: and every thing, whereon he sitteth, shall be unclean."],
    ['Leviticus','Leviticus','15','05','5',"And whosoever toucheth his bed shall wash his clothes, and bathe \emph{himself} in water, and be unclean until the even."],
    ['Leviticus','Leviticus','15','06','6',"And he that sitteth on \emph{any} thing whereon he sat that hath the issue shall wash his clothes, and bathe \emph{himself} in water, and be unclean until the even."],
    ['Leviticus','Leviticus','15','07','7',"And he that toucheth the flesh of him that hath the issue shall wash his clothes, and bathe \emph{himself} in water, and be unclean until the even."],
    ['Leviticus','Leviticus','15','08','8',"And if he that hath the issue spit upon him that is clean; then he shall wash his clothes, and bathe \emph{himself} in water, and be unclean until the even."],
    ['Leviticus','Leviticus','15','09','9',"And what saddle soever he rideth upon that hath the issue shall be unclean."],
    ['Leviticus','Leviticus','15','10','10',"And whosoever toucheth any thing that was under him shall be unclean until the even: and he that beareth \emph{any} \emph{of} those things shall wash his clothes, and bathe \emph{himself} in water, and be unclean until the even."],
    ['Leviticus','Leviticus','15','11','11',"And whomsoever he toucheth that hath the issue, and hath not rinsed his hands in water, he shall wash his clothes, and bathe \emph{himself} in water, and be unclean until the even."],
    ['Leviticus','Leviticus','15','12','12',"And the vessel of earth, that he toucheth which hath the issue, shall be broken: and every vessel of wood shall be rinsed in water."],
    ['Leviticus','Leviticus','15','13','13',"And when he that hath an issue is cleansed of his issue; then he shall number to himself seven days for his cleansing, and wash his clothes, and bathe his flesh in running water, and shall be clean."],
    ['Leviticus','Leviticus','15','14','14',"And on the eighth day he shall take to him two turtledoves, or two young pigeons, and come before the LORD unto the door of the tabernacle of the congregation, and give them unto the priest:"],
    ['Leviticus','Leviticus','15','15','15',"And the priest shall offer them, the one \emph{for} a sin offering, and the other \emph{for} a burnt offering; and the priest shall make an atonement for him before the LORD for his issue."],
    ['Leviticus','Leviticus','15','16','16',"And if any man's seed of copulation go out from him, then he shall wash all his flesh in water, and be unclean until the even."],
    ['Leviticus','Leviticus','15','17','17',"And every garment, and every skin, whereon is the seed of copulation, shall be washed with water, and be unclean until the even."],
    ['Leviticus','Leviticus','15','18','18',"The woman also with whom man shall lie \emph{with} seed of copulation, they shall \emph{both} bathe \emph{themselves} in water, and be unclean until the even."],
    ['Leviticus','Leviticus','15','19','19',"And if a woman have an issue, \emph{and} her issue in her flesh be blood, she shall be put apart seven days: and whosoever toucheth her shall be unclean until the even."],
    ['Leviticus','Leviticus','15','20','20',"And every thing that she lieth upon in her separation shall be unclean: every thing also that she sitteth upon shall be unclean."],
    ['Leviticus','Leviticus','15','21','21',"And whosoever toucheth her bed shall wash his clothes, and bathe \emph{himself} in water, and be unclean until the even."],
    ['Leviticus','Leviticus','15','22','22',"And whosoever toucheth any thing that she sat upon shall wash his clothes, and bathe \emph{himself} in water, and be unclean until the even."],
    ['Leviticus','Leviticus','15','23','23',"And if it \emph{be} on \emph{her} bed, or on any thing whereon she sitteth, when he toucheth it, he shall be unclean until the even."],
    ['Leviticus','Leviticus','15','24','24',"And if any man lie with her at all, and her flowers be upon him, he shall be unclean seven days; and all the bed whereon he lieth shall be unclean."],
    ['Leviticus','Leviticus','15','25','25',"And if a woman have an issue of her blood many days out of the time of her separation, or if it run beyond the time of her separation; all the days of the issue of her uncleanness shall be as the days of her separation: she \emph{shall} \emph{be} unclean."],
    ['Leviticus','Leviticus','15','26','26',"Every bed whereon she lieth all the days of her issue shall be unto her as the bed of her separation: and whatsoever she sitteth upon shall be unclean, as the uncleanness of her separation."],
    ['Leviticus','Leviticus','15','27','27',"And whosoever toucheth those things shall be unclean, and shall wash his clothes, and bathe \emph{himself} in water, and be unclean until the even."],
    ['Leviticus','Leviticus','15','28','28',"But if she be cleansed of her issue, then she shall number to herself seven days, and after that she shall be clean."],
    ['Leviticus','Leviticus','15','29','29',"And on the eighth day she shall take unto her two turtles, or two young pigeons, and bring them unto the priest, to the door of the tabernacle of the congregation."],
    ['Leviticus','Leviticus','15','30','30',"And the priest shall offer the one \emph{for} a sin offering, and the other \emph{for} a burnt offering; and the priest shall make an atonement for her before the LORD for the issue of her uncleanness."],
    ['Leviticus','Leviticus','15','31','31',"Thus shall ye separate the children of Israel from their uncleanness; that they die not in their uncleanness, when they defile my tabernacle that \emph{is} among them."],
    ['Leviticus','Leviticus','15','32','32',"This \emph{is} the law of him that hath an issue, and \emph{of} \emph{him} whose seed goeth from him, and is defiled therewith;"],
    ['Leviticus','Leviticus','15','33','33',"And of her that is sick of her flowers, and of him that hath an issue, of the man, and of the woman, and of him that lieth with her that is unclean."]]

Leviticus16 = [
    ['Leviticus','Lev','16','001','1',"And the LORD spake unto Moses after the death of the two sons of Aaron, when they offered before the LORD, and died;"],
['Leviticus','Lev','16','002','2',"And the LORD said unto Moses, Speak unto Aaron thy brother, that he come not at all times into the holy \emph{place} within the vail before the mercy seat, which \emph{is} upon the ark; that he die not: for I will appear in the cloud upon the mercy seat."],
['Leviticus','Lev','16','003','3',"Thus shall Aaron come into the holy \emph{place}: with a young bullock for a sin offering, and a ram for a burnt offering."],
['Leviticus','Lev','16','004','4',"He shall put on the holy linen coat, and he shall have the linen breeches upon his flesh, and shall be girded with a linen girdle, and with the linen mitre shall he be attired: these \emph{are} holy garments; therefore shall he wash his flesh in water, and \emph{so} put them on."],
['Leviticus','Lev','16','005','5',"And he shall take of the congregation of the children of Israel two kids of the goats for a sin offering, and one ram for a burnt offering."],
['Leviticus','Lev','16','006','6',"And Aaron shall offer his bullock of the sin offering, which \emph{is} for himself, and make an atonement for himself, and for his house."],
['Leviticus','Lev','16','007','7',"And he shall take the two goats, and present them before the LORD \emph{at} the door of the tabernacle of the congregation."],
['Leviticus','Lev','16','008','8',"And Aaron shall cast lots upon the two goats; one lot for the LORD, and the other lot for the scapegoat."],
['Leviticus','Lev','16','009','9',"And Aaron shall bring the goat upon which the LORD'S lot fell, and offer him \emph{for} a sin offering."],
['Leviticus','Lev','16','010','10',"But the goat, on which the lot fell to be the scapegoat, shall be presented alive before the LORD, to make an atonement with him, \emph{and} to let him go for a scapegoat into the wilderness."],
['Leviticus','Lev','16','011','11',"And Aaron shall bring the bullock of the sin offering, which \emph{is} for himself, and shall make an atonement for himself, and for his house, and shall kill the bullock of the sin offering which \emph{is} for himself:"],
['Leviticus','Lev','16','012','12',"And he shall take a censer full of burning coals of fire from off the altar before the LORD, and his hands full of sweet incense beaten small, and bring \emph{it} within the vail:"],
['Leviticus','Lev','16','013','13',"And he shall put the incense upon the fire before the LORD, that the cloud of the incense may cover the mercy seat that \emph{is} upon the testimony, that he die not:"],
['Leviticus','Lev','16','014','14',"And he shall take of the blood of the bullock, and sprinkle \emph{it} with his finger upon the mercy seat eastward; and before the mercy seat shall he sprinkle of the blood with his finger seven times."],
['Leviticus','Lev','16','015','15',"Then shall he kill the goat of the sin offering, that \emph{is} for the people, and bring his blood within the vail, and do with that blood as he did with the blood of the bullock, and sprinkle it upon the mercy seat, and before the mercy seat:"],
['Leviticus','Lev','16','016','16',"And he shall make an atonement for the holy \emph{place}, because of the uncleanness of the children of Israel, and because of their transgressions in all their sins: and so shall he do for the tabernacle of the congregation, that remaineth among them in the midst of their uncleanness."],
['Leviticus','Lev','16','017','17',"And there shall be no man in the tabernacle of the congregation when he goeth in to make an atonement in the holy \emph{place}, until he come out, and have made an atonement for himself, and for his household, and for all the congregation of Israel."],
['Leviticus','Lev','16','018','18',"And he shall go out unto the altar that \emph{is} before the LORD, and make an atonement for it; and shall take of the blood of the bullock, and of the blood of the goat, and put \emph{it} upon the horns of the altar round about."],
['Leviticus','Lev','16','019','19',"And he shall sprinkle of the blood upon it with his finger seven times, and cleanse it, and hallow it from the uncleanness of the children of Israel."],
['Leviticus','Lev','16','020','20',"And when he hath made an end of reconciling the holy \emph{place}, and the tabernacle of the congregation, and the altar, he shall bring the live goat:"],
['Leviticus','Lev','16','021','21',"And Aaron shall lay both his hands upon the head of the live goat, and confess over him all the iniquities of the children of Israel, and all their transgressions in all their sins, putting them upon the head of the goat, and shall send \emph{him} away by the hand of a fit man into the wilderness:"],
['Leviticus','Lev','16','022','22',"And the goat shall bear upon him all their iniquities unto a land not inhabited: and he shall let go the goat in the wilderness."],
['Leviticus','Lev','16','023','23',"And Aaron shall come into the tabernacle of the congregation, and shall put off the linen garments, which he put on when he went into the holy \emph{place}, and shall leave them there:"],
['Leviticus','Lev','16','024','24',"And he shall wash his flesh with water in the holy place, and put on his garments, and come forth, and offer his burnt offering, and the burnt offering of the people, and make an atonement for himself, and for the people."],
['Leviticus','Lev','16','025','25',"And the fat of the sin offering shall he burn upon the altar."],
['Leviticus','Lev','16','026','26',"And he that let go the goat for the scapegoat shall wash his clothes, and bathe his flesh in water, and afterward come into the camp."],
['Leviticus','Lev','16','027','27',"And the bullock \emph{for} the sin offering, and the goat \emph{for} the sin offering, whose blood was brought in to make atonement in the holy \emph{place}, shall \emph{one} carry forth without the camp; and they shall burn in the fire their skins, and their flesh, and their dung."],
['Leviticus','Lev','16','028','28',"And he that burneth them shall wash his clothes, and bathe his flesh in water, and afterward he shall come into the camp."],
['Leviticus','Lev','16','029','29',"And \emph{this} shall be a statute for ever unto you: \emph{that} in the seventh month, on the tenth \emph{day} of the month, ye shall afflict your souls, and do no work at all, \emph{whether} \emph{it} \emph{be} one of your own country, or a stranger that sojourneth among you:"],
['Leviticus','Lev','16','030','30',"For on that day shall \emph{the} \emph{priest} make an atonement for you, to cleanse you, \emph{that} ye may be clean from all your sins before the LORD."],
['Leviticus','Lev','16','031','31',"It \emph{shall} \emph{be} a sabbath of rest unto you, and ye shall afflict your souls, by a statute for ever."],
['Leviticus','Lev','16','032','32',"And the priest, whom he shall anoint, and whom he shall consecrate to minister in the priest's office in his father's stead, shall make the atonement, and shall put on the linen clothes, \emph{even} the holy garments:"],
['Leviticus','Lev','16','033','33',"And he shall make an atonement for the holy sanctuary, and he shall make an atonement for the tabernacle of the congregation, and for the altar, and he shall make an atonement for the priests, and for all the people of the congregation."],
['Leviticus','Lev','16','034','34',"And this shall be an everlasting statute unto you, to make an atonement for the children of Israel for all their sins once a year. And he did as the LORD commanded Moses."]
]

Leviticus17 = [
    ['Leviticus','Lev','17','001','1',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','17','002','2',"Speak unto Aaron, and unto his sons, and unto all the children of Israel, and say unto them; This \emph{is} the thing which the LORD hath commanded, saying,"],
['Leviticus','Lev','17','003','3',"What man soever \emph{there} \emph{be} of the house of Israel, that killeth an ox, or lamb, or goat, in the camp, or that killeth \emph{it} out of the camp,"],
['Leviticus','Lev','17','004','4',"And bringeth it not unto the door of the tabernacle of the congregation, to offer an offering unto the LORD before the tabernacle of the LORD; blood shall be imputed unto that man; he hath shed blood; and that man shall be cut off from among his people:"],
['Leviticus','Lev','17','005','5',"To the end that the children of Israel may bring their sacrifices, which they offer in the open field, even that they may bring them unto the LORD, unto the door of the tabernacle of the congregation, unto the priest, and offer them \emph{for} peace offerings unto the LORD."],
['Leviticus','Lev','17','006','6',"And the priest shall sprinkle the blood upon the altar of the LORD \emph{at} the door of the tabernacle of the congregation, and burn the fat for a sweet savour unto the LORD."],
['Leviticus','Lev','17','007','7',"And they shall no more offer their sacrifices unto devils, after whom they have gone a whoring. This shall be a statute for ever unto them throughout their generations."],
['Leviticus','Lev','17','008','8',"And thou shalt say unto them, Whatsoever man \emph{there} \emph{be} of the house of Israel, or of the strangers which sojourn among you, that offereth a burnt offering or sacrifice,"],
['Leviticus','Lev','17','009','9',"And bringeth it not unto the door of the tabernacle of the congregation, to offer it unto the LORD; even that man shall be cut off from among his people."],
['Leviticus','Lev','17','010','10',"And whatsoever man \emph{there} \emph{be} of the house of Israel, or of the strangers that sojourn among you, that eateth any manner of blood; I will even set my face against that soul that eateth blood, and will cut him off from among his people."],
['Leviticus','Lev','17','011','11',"For the life of the flesh \emph{is} in the blood: and I have given it to you upon the altar to make an atonement for your souls: for it \emph{is} the blood \emph{that} maketh an atonement for the soul."],
['Leviticus','Lev','17','012','12',"Therefore I said unto the children of Israel, No soul of you shall eat blood, neither shall any stranger that sojourneth among you eat blood."],
['Leviticus','Lev','17','013','13',"And whatsoever man \emph{there} \emph{be} of the children of Israel, or of the strangers that sojourn among you, which hunteth and catcheth any beast or fowl that may be eaten; he shall even pour out the blood thereof, and cover it with dust."],
['Leviticus','Lev','17','014','14',"For \emph{it} \emph{is} the life of all flesh; the blood of it \emph{is} for the life thereof: therefore I said unto the children of Israel, Ye shall eat the blood of no manner of flesh: for the life of all flesh \emph{is} the blood thereof: whosoever eateth it shall be cut off."],
['Leviticus','Lev','17','015','15',"And every soul that eateth that which died \emph{of} \emph{itself}, or that which was torn \emph{with} \emph{beasts,} \emph{whether} \emph{it} \emph{be} one of your own country, or a stranger, he shall both wash his clothes, and bathe \emph{himself} in water, and be unclean until the even: then shall he be clean."],
['Leviticus','Lev','17','016','16',"But if he wash \emph{them} not, nor bathe his flesh; then he shall bear his iniquity."]
]

Leviticus18 = [
    ['Leviticus','Lev','18','001','1',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','18','002','2',"Speak unto the children of Israel, and say unto them, I am the LORD your God."],
['Leviticus','Lev','18','003','3',"After the doings of the land of Egypt, wherein ye dwelt, shall ye not do: and after the doings of the land of Canaan, whither I bring you, shall ye not do: neither shall ye walk in their ordinances."],
['Leviticus','Lev','18','004','4',"Ye shall do my judgments, and keep mine ordinances, to walk therein: I \emph{am} the LORD your God."],
['Leviticus','Lev','18','005','5',"Ye shall therefore keep my statutes, and my judgments: which if a man do, he shall live in them: I \emph{am} the LORD."],
['Leviticus','Lev','18','006','6',"None of you shall approach to any that is near of kin to him, to uncover \emph{their} nakedness: I \emph{am} the LORD."],
['Leviticus','Lev','18','007','7',"The nakedness of thy father, or the nakedness of thy mother, shalt thou not uncover: she \emph{is} thy mother; thou shalt not uncover her nakedness."],
['Leviticus','Lev','18','008','8',"The nakedness of thy father's wife shalt thou not uncover: it \emph{is} thy father's nakedness."],
['Leviticus','Lev','18','009','9',"The nakedness of thy sister, the daughter of thy father, or daughter of thy mother, \emph{whether} \emph{she} \emph{be} born at home, or born abroad, \emph{even} their nakedness thou shalt not uncover."],
['Leviticus','Lev','18','010','10',"The nakedness of thy son's daughter, or of thy daughter's daughter, \emph{even} their nakedness thou shalt not uncover: for their's \emph{is} thine own nakedness."],
['Leviticus','Lev','18','011','11',"The nakedness of thy father's wife's daughter, begotten of thy father, she \emph{is} thy sister, thou shalt not uncover her nakedness."],
['Leviticus','Lev','18','012','12',"Thou shalt not uncover the nakedness of thy father's sister: she \emph{is} thy father's near kinswoman."],
['Leviticus','Lev','18','013','13',"Thou shalt not uncover the nakedness of thy mother's sister: for she \emph{is} thy mother's near kinswoman."],
['Leviticus','Lev','18','014','14',"Thou shalt not uncover the nakedness of thy father's brother, thou shalt not approach to his wife: she \emph{is} thine aunt."],
['Leviticus','Lev','18','015','15',"Thou shalt not uncover the nakedness of thy daughter in law: she \emph{is} thy son's wife; thou shalt not uncover her nakedness."],
['Leviticus','Lev','18','016','16',"Thou shalt not uncover the nakedness of thy brother's wife: it \emph{is} thy brother's nakedness."],
['Leviticus','Lev','18','017','17',"Thou shalt not uncover the nakedness of a woman and her daughter, neither shalt thou take her son's daughter, or her daughter's daughter, to uncover her nakedness; \emph{for} they \emph{are} her near kinswomen: it \emph{is} wickedness."],
['Leviticus','Lev','18','018','18',"Neither shalt thou take a wife to her sister, to vex \emph{her}, to uncover her nakedness, beside the other in her life \emph{time}."],
['Leviticus','Lev','18','019','19',"Also thou shalt not approach unto a woman to uncover her nakedness, as long as she is put apart for her uncleanness."],
['Leviticus','Lev','18','020','20',"Moreover thou shalt not lie carnally with thy neighbour's wife, to defile thyself with her."],
['Leviticus','Lev','18','021','21',"And thou shalt not let any of thy seed pass through \emph{the} \emph{fire} to Molech, neither shalt thou profane the name of thy God: I \emph{am} the LORD."],
['Leviticus','Lev','18','022','22',"Thou shalt not lie with mankind, as with womankind: it \emph{is} abomination."],
['Leviticus','Lev','18','023','23',"Neither shalt thou lie with any beast to defile thyself therewith: neither shall any woman stand before a beast to lie down thereto: it \emph{is} confusion."],
['Leviticus','Lev','18','024','24',"Defile not ye yourselves in any of these things: for in all these the nations are defiled which I cast out before you:"],
['Leviticus','Lev','18','025','25',"And the land is defiled: therefore I do visit the iniquity thereof upon it, and the land itself vomiteth out her inhabitants."],
['Leviticus','Lev','18','026','26',"Ye shall therefore keep my statutes and my judgments, and shall not commit \emph{any} of these abominations; \emph{neither} any of your own nation, nor any stranger that sojourneth among you:"],
['Leviticus','Lev','18','027','27',"(For all these abominations have the men of the land done, which \emph{were} before you, and the land is defiled;)"],
['Leviticus','Lev','18','028','28',"That the land spue not you out also, when ye defile it, as it spued out the nations that \emph{were} before you."],
['Leviticus','Lev','18','029','29',"For whosoever shall commit any of these abominations, even the souls that commit \emph{them} shall be cut off from among their people."],
['Leviticus','Lev','18','030','30',"Therefore shall ye keep mine ordinance, that \emph{ye} commit not \emph{any} \emph{one} of these abominable customs, which were committed before you, and that ye defile not yourselves therein: I \emph{am} the LORD your God."]
]

Leviticus19 = [
    ['Leviticus','Lev','19','001','1',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','19','002','2',"Speak unto all the congregation of the children of Israel, and say unto them, Ye shall be holy: for I the LORD your God \emph{am} holy."],
['Leviticus','Lev','19','003','3',"Ye shall fear every man his mother, and his father, and keep my sabbaths: I \emph{am} the LORD your God."],
['Leviticus','Lev','19','004','4',"Turn ye not unto idols, nor make to yourselves molten gods: I \emph{am} the LORD your God."],
['Leviticus','Lev','19','005','5',"And if ye offer a sacrifice of peace offerings unto the LORD, ye shall offer it at your own will."],
['Leviticus','Lev','19','006','6',"It shall be eaten the same day ye offer it, and on the morrow: and if ought remain until the third day, it shall be burnt in the fire."],
['Leviticus','Lev','19','007','7',"And if it be eaten at all on the third day, it \emph{is} abominable; it shall not be accepted."],
['Leviticus','Lev','19','008','8',"Therefore \emph{every} \emph{one} that eateth it shall bear his iniquity, because he hath profaned the hallowed thing of the LORD: and that soul shall be cut off from among his people."],
['Leviticus','Lev','19','009','9',"And when ye reap the harvest of your land, thou shalt not wholly reap the corners of thy field, neither shalt thou gather the gleanings of thy harvest."],
['Leviticus','Lev','19','010','10',"And thou shalt not glean thy vineyard, neither shalt thou gather \emph{every} grape of thy vineyard; thou shalt leave them for the poor and stranger: I \emph{am} the LORD your God."],
['Leviticus','Lev','19','011','11',"Ye shall not steal, neither deal falsely, neither lie one to another."],
['Leviticus','Lev','19','012','12',"And ye shall not swear by my name falsely, neither shalt thou profane the name of thy God: I \emph{am} the LORD."],
['Leviticus','Lev','19','013','13',"Thou shalt not defraud thy neighbour, neither rob \emph{him}: the wages of him that is hired shall not abide with thee all night until the morning."],
['Leviticus','Lev','19','014','14',"Thou shalt not curse the deaf, nor put a stumblingblock before the blind, but shalt fear thy God: I \emph{am} the LORD."],
['Leviticus','Lev','19','015','15',"Ye shall do no unrighteousness in judgment: thou shalt not respect the person of the poor, nor honour the person of the mighty: \emph{but} in righteousness shalt thou judge thy neighbour."],
['Leviticus','Lev','19','016','16',"Thou shalt not go up and down \emph{as} a talebearer among thy people: neither shalt thou stand against the blood of thy neighbour: I \emph{am} the LORD."],
['Leviticus','Lev','19','017','17',"Thou shalt not hate thy brother in thine heart: thou shalt in any wise rebuke thy neighbour, and not suffer sin upon him."],
['Leviticus','Lev','19','018','18',"Thou shalt not avenge, nor bear any grudge against the children of thy people, but thou shalt love thy neighbour as thyself: I \emph{am} the LORD."],
['Leviticus','Lev','19','019','19',"Ye shall keep my statutes. Thou shalt not let thy cattle gender with a diverse kind: thou shalt not sow thy field with mingled seed: neither shall a garment mingled of linen and woollen come upon thee."],
['Leviticus','Lev','19','020','20',"And whosoever lieth carnally with a woman, that \emph{is} a bondmaid, betrothed to an husband, and not at all redeemed, nor freedom given her; she shall be scourged; they shall not be put to death, because she was not free."],
['Leviticus','Lev','19','021','21',"And he shall bring his trespass offering unto the LORD, unto the door of the tabernacle of the congregation, \emph{even} a ram for a trespass offering."],
['Leviticus','Lev','19','022','22',"And the priest shall make an atonement for him with the ram of the trespass offering before the LORD for his sin which he hath done: and the sin which he hath done shall be forgiven him."],
['Leviticus','Lev','19','023','23',"And when ye shall come into the land, and shall have planted all manner of trees for food, then ye shall count the fruit thereof as uncircumcised: three years shall it be as uncircumcised unto you: it shall not be eaten of."],
['Leviticus','Lev','19','024','24',"But in the fourth year all the fruit thereof shall be holy to praise the LORD \emph{withal}."],
['Leviticus','Lev','19','025','25',"And in the fifth year shall ye eat of the fruit thereof, that it may yield unto you the increase thereof: I \emph{am} the LORD your God."],
['Leviticus','Lev','19','026','26',"Ye shall not eat \emph{any} \emph{thing} with the blood: neither shall ye use enchantment, nor observe times."],
['Leviticus','Lev','19','027','27',"Ye shall not round the corners of your heads, neither shalt thou mar the corners of thy beard."],
['Leviticus','Lev','19','028','28',"Ye shall not make any cuttings in your flesh for the dead, nor print any marks upon you: I \emph{am} the LORD."],
['Leviticus','Lev','19','029','29',"Do not prostitute thy daughter, to cause her to be a whore; lest the land fall to whoredom, and the land become full of wickedness."],
['Leviticus','Lev','19','030','30',"Ye shall keep my sabbaths, and reverence my sanctuary: I \emph{am} the LORD."],
['Leviticus','Lev','19','031','31',"Regard not them that have familiar spirits, neither seek after wizards, to be defiled by them: I \emph{am} the LORD your God."],
['Leviticus','Lev','19','032','32',"Thou shalt rise up before the hoary head, and honour the face of the old man, and fear thy God: I \emph{am} the LORD."],
['Leviticus','Lev','19','033','33',"And if a stranger sojourn with thee in your land, ye shall not vex him."],
['Leviticus','Lev','19','034','34',"\emph{But} the stranger that dwelleth with you shall be unto you as one born among you, and thou shalt love him as thyself; for ye were strangers in the land of Egypt: I \emph{am} the LORD your God."],
['Leviticus','Lev','19','035','35',"Ye shall do no unrighteousness in judgment, in meteyard, in weight, or in measure."],
['Leviticus','Lev','19','036','36',"Just balances, just weights, a just ephah, and a just hin, shall ye have: I \emph{am} the LORD your God, which brought you out of the land of Egypt."],
['Leviticus','Lev','19','037','37',"Therefore shall ye observe all my statutes, and all my judgments, and do them: I \emph{am} the LORD."]
]

Leviticus20 = [
    ['Leviticus','Lev','20','001','1',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','20','002','2',"Again, thou shalt say to the children of Israel, Whosoever \emph{he} \emph{be} of the children of Israel, or of the strangers that sojourn in Israel, that giveth \emph{any} of his seed unto Molech; he shall surely be put to death: the people of the land shall stone him with stones."],
['Leviticus','Lev','20','003','3',"And I will set my face against that man, and will cut him off from among his people; because he hath given of his seed unto Molech, to defile my sanctuary, and to profane my holy name."],
['Leviticus','Lev','20','004','4',"And if the people of the land do any ways hide their eyes from the man, when he giveth of his seed unto Molech, and kill him not:"],
['Leviticus','Lev','20','005','5',"Then I will set my face against that man, and against his family, and will cut him off, and all that go a whoring after him, to commit whoredom with Molech, from among their people."],
['Leviticus','Lev','20','006','6',"And the soul that turneth after such as have familiar spirits, and after wizards, to go a whoring after them, I will even set my face against that soul, and will cut him off from among his people."],
['Leviticus','Lev','20','007','7',"Sanctify yourselves therefore, and be ye holy: for I \emph{am} the LORD your God."],
['Leviticus','Lev','20','008','8',"And ye shall keep my statutes, and do them: I \emph{am} the LORD which sanctify you."],
['Leviticus','Lev','20','009','9',"For every one that curseth his father or his mother shall be surely put to death: he hath cursed his father or his mother; his blood \emph{shall} \emph{be} upon him."],
['Leviticus','Lev','20','010','10',"And the man that committeth adultery with \emph{another} man's wife, \emph{even} \emph{he} that committeth adultery with his neighbour's wife, the adulterer and the adulteress shall surely be put to death."],
['Leviticus','Lev','20','011','11',"And the man that lieth with his father's wife hath uncovered his father's nakedness: both of them shall surely be put to death; their blood \emph{shall} \emph{be} upon them."],
['Leviticus','Lev','20','012','12',"And if a man lie with his daughter in law, both of them shall surely be put to death: they have wrought confusion; their blood \emph{shall} \emph{be} upon them."],
['Leviticus','Lev','20','013','13',"If a man also lie with mankind, as he lieth with a woman, both of them have committed an abomination: they shall surely be put to death; their blood \emph{shall} \emph{be} upon them."],
['Leviticus','Lev','20','014','14',"And if a man take a wife and her mother, it \emph{is} wickedness: they shall be burnt with fire, both he and they; that there be no wickedness among you."],
['Leviticus','Lev','20','015','15',"And if a man lie with a beast, he shall surely be put to death: and ye shall slay the beast."],
['Leviticus','Lev','20','016','16',"And if a woman approach unto any beast, and lie down thereto, thou shalt kill the woman, and the beast: they shall surely be put to death; their blood \emph{shall} \emph{be} upon them."],
['Leviticus','Lev','20','017','17',"And if a man shall take his sister, his father's daughter, or his mother's daughter, and see her nakedness, and she see his nakedness; it \emph{is} a wicked thing; and they shall be cut off in the sight of their people: he hath uncovered his sister's nakedness; he shall bear his iniquity."],
['Leviticus','Lev','20','018','18',"And if a man shall lie with a woman having her sickness, and shall uncover her nakedness; he hath discovered her fountain, and she hath uncovered the fountain of her blood: and both of them shall be cut off from among their people."],
['Leviticus','Lev','20','019','19',"And thou shalt not uncover the nakedness of thy mother's sister, nor of thy father's sister: for he uncovereth his near kin: they shall bear their iniquity."],
['Leviticus','Lev','20','020','20',"And if a man shall lie with his uncle's wife, he hath uncovered his uncle's nakedness: they shall bear their sin; they shall die childless."],
['Leviticus','Lev','20','021','21',"And if a man shall take his brother's wife, it \emph{is} an unclean thing: he hath uncovered his brother's nakedness; they shall be childless."],
['Leviticus','Lev','20','022','22',"Ye shall therefore keep all my statutes, and all my judgments, and do them: that the land, whither I bring you to dwell therein, spue you not out."],
['Leviticus','Lev','20','023','23',"And ye shall not walk in the manners of the nation, which I cast out before you: for they committed all these things, and therefore I abhorred them."],
['Leviticus','Lev','20','024','24',"But I have said unto you, Ye shall inherit their land, and I will give it unto you to possess it, a land that floweth with milk and honey: I \emph{am} the LORD your God, which have separated you from \emph{other} people."],
['Leviticus','Lev','20','025','25',"Ye shall therefore put difference between clean beasts and unclean, and between unclean fowls and clean: and ye shall not make your souls abominable by beast, or by fowl, or by any manner of living thing that creepeth on the ground, which I have separated from you as unclean."],
['Leviticus','Lev','20','026','26',"And ye shall be holy unto me: for I the LORD \emph{am} holy, and have severed you from \emph{other} people, that ye should be mine."],
['Leviticus','Lev','20','027','27',"A man also or woman that hath a familiar spirit, or that is a wizard, shall surely be put to death: they shall stone them with stones: their blood \emph{shall} \emph{be} upon them."]
]

Leviticus21 = [
    ['Leviticus','Lev','21','001','1',"And the LORD said unto Moses, Speak unto the priests the sons of Aaron, and say unto them, There shall none be defiled for the dead among his people:"],
['Leviticus','Lev','21','002','2',"But for his kin, that is near unto him, \emph{that} \emph{is}, for his mother, and for his father, and for his son, and for his daughter, and for his brother,"],
['Leviticus','Lev','21','003','3',"And for his sister a virgin, that is nigh unto him, which hath had no husband; for her may he be defiled."],
['Leviticus','Lev','21','004','4',"\emph{But} he shall not defile himself, \emph{being} a chief man among his people, to profane himself."],
['Leviticus','Lev','21','005','5',"They shall not make baldness upon their head, neither shall they shave off the corner of their beard, nor make any cuttings in their flesh."],
['Leviticus','Lev','21','006','6',"They shall be holy unto their God, and not profane the name of their God: for the offerings of the LORD made by fire, \emph{and} the bread of their God, they do offer: therefore they shall be holy."],
['Leviticus','Lev','21','007','7',"They shall not take a wife \emph{that} \emph{is} a whore, or profane; neither shall they take a woman put away from her husband: for he \emph{is} holy unto his God."],
['Leviticus','Lev','21','008','8',"Thou shalt sanctify him therefore; for he offereth the bread of thy God: he shall be holy unto thee: for I the LORD, which sanctify you, \emph{am} holy."],
['Leviticus','Lev','21','009','9',"And the daughter of any priest, if she profane herself by playing the whore, she profaneth her father: she shall be burnt with fire."],
['Leviticus','Lev','21','010','10',"And \emph{he} \emph{that} \emph{is} the high priest among his brethren, upon whose head the anointing oil was poured, and that is consecrated to put on the garments, shall not uncover his head, nor rend his clothes;"],
['Leviticus','Lev','21','011','11',"Neither shall he go in to any dead body, nor defile himself for his father, or for his mother;"],
['Leviticus','Lev','21','012','12',"Neither shall he go out of the sanctuary, nor profane the sanctuary of his God; for the crown of the anointing oil of his God \emph{is} upon him: I \emph{am} the LORD."],
['Leviticus','Lev','21','013','13',"And he shall take a wife in her virginity."],
['Leviticus','Lev','21','014','14',"A widow, or a divorced woman, or profane, \emph{or} an harlot, these shall he not take: but he shall take a virgin of his own people to wife."],
['Leviticus','Lev','21','015','15',"Neither shall he profane his seed among his people: for I the LORD do sanctify him."],
['Leviticus','Lev','21','016','16',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','21','017','17',"Speak unto Aaron, saying, Whosoever \emph{he} \emph{be} of thy seed in their generations that hath \emph{any} blemish, let him not approach to offer the bread of his God."],
['Leviticus','Lev','21','018','18',"For whatsoever man \emph{he} \emph{be} that hath a blemish, he shall not approach: a blind man, or a lame, or he that hath a flat nose, or any thing superfluous,"],
['Leviticus','Lev','21','019','19',"Or a man that is brokenfooted, or brokenhanded,"],
['Leviticus','Lev','21','020','20',"Or crookbackt, or a dwarf, or that hath a blemish in his eye, or be scurvy, or scabbed, or hath his stones broken;"],
['Leviticus','Lev','21','021','21',"No man that hath a blemish of the seed of Aaron the priest shall come nigh to offer the offerings of the LORD made by fire: he hath a blemish; he shall not come nigh to offer the bread of his God."],
['Leviticus','Lev','21','022','22',"He shall eat the bread of his God, \emph{both} of the most holy, and of the holy."],
['Leviticus','Lev','21','023','23',"Only he shall not go in unto the vail, nor come nigh unto the altar, because he hath a blemish; that he profane not my sanctuaries: for I the LORD do sanctify them."],
['Leviticus','Lev','21','024','24',"And Moses told \emph{it} unto Aaron, and to his sons, and unto all the children of Israel."]
]

Leviticus22 = [
    ['Leviticus','Lev','22','001','1',"And the LORD spake unto Moses, saying,"],
    ['Leviticus','Lev','22','002','2',"Speak unto Aaron and to his sons, that they separate themselves from the holy things of the children of Israel, and that they profane not my holy name \emph{in} \emph{those} \emph{things} which they hallow unto me: I \emph{am} the LORD."],
    ['Leviticus','Lev','22','003','3',"Say unto them, Whosoever \emph{he} \emph{be} of all your seed among your generations, that goeth unto the holy things, which the children of Israel hallow unto the LORD, having his uncleanness upon him, that soul shall be cut off from my presence: I \emph{am} the LORD."],
    ['Leviticus','Lev','22','004','4',"What man soever of the seed of Aaron \emph{is} a leper, or hath a running issue; he shall not eat of the holy things, until he be clean. And whoso toucheth any thing \emph{that} \emph{is} unclean \emph{by} the dead, or a man whose seed goeth from him;"],
    ['Leviticus','Lev','22','005','5',"Or whosoever toucheth any creeping thing, whereby he may be made unclean, or a man of whom he may take uncleanness, whatsoever uncleanness he hath;"],
    ['Leviticus','Lev','22','006','6',"The soul which hath touched any such shall be unclean until even, and shall not eat of the holy things, unless he wash his flesh with water."],
    ['Leviticus','Lev','22','007','7',"And when the sun is down, he shall be clean, and shall afterward eat of the holy things; because it \emph{is} his food."],
    ['Leviticus','Lev','22','008','8',"That which dieth of itself, or is torn \emph{with} \emph{beasts}, he shall not eat to defile himself therewith: I \emph{am} the LORD."],
    ['Leviticus','Lev','22','009','9',"They shall therefore keep mine ordinance, lest they bear sin for it, and die therefore, if they profane it: I the LORD do sanctify them."],
    ['Leviticus','Lev','22','010','10',"There shall no stranger eat \emph{of} the holy thing: a sojourner of the priest, or an hired servant, shall not eat \emph{of} the holy thing."],
    ['Leviticus','Lev','22','011','11',"But if the priest buy \emph{any} soul with his money, he shall eat of it, and he that is born in his house: they shall eat of his meat."],
    ['Leviticus','Lev','22','012','12',"If the priest's daughter also be \emph{married} unto a stranger, she may not eat of an offering of the holy things."],
    ['Leviticus','Lev','22','013','13',"But if the priest's daughter be a widow, or divorced, and have no child, and is returned unto her father's house, as in her youth, she shall eat of her father's meat: but there shall no stranger eat thereof."],
    ['Leviticus','Lev','22','014','14',"And if a man eat \emph{of} the holy thing unwittingly, then he shall put the fifth \emph{part} thereof unto it, and shall give \emph{it} unto the priest with the holy thing."],
    ['Leviticus','Lev','22','015','15',"And they shall not profane the holy things of the children of Israel, which they offer unto the LORD;"],
    ['Leviticus','Lev','22','016','16',"Or suffer them to bear the iniquity of trespass, when they eat their holy things: for I the LORD do sanctify them."],
    ['Leviticus','Lev','22','017','17',"And the LORD spake unto Moses, saying,"],
    ['Leviticus','Lev','22','018','18',"Speak unto Aaron, and to his sons, and unto all the children of Israel, and say unto them, Whatsoever \emph{he} \emph{be} of the house of Israel, or of the strangers in Israel, that will offer his oblation for all his vows, and for all his freewill offerings, which they will offer unto the LORD for a burnt offering;"],
    ['Leviticus','Lev','22','019','19',"\emph{Ye} \emph{shall} \emph{offer} at your own will a male without blemish, of the beeves, of the sheep, or of the goats."],
    ['Leviticus','Lev','22','020','20',"\emph{But} whatsoever hath a blemish, \emph{that} shall ye not offer: for it shall not be acceptable for you."],
    ['Leviticus','Lev','22','021','21',"And whosoever offereth a sacrifice of peace offerings unto the LORD to accomplish \emph{his} vow, or a freewill offering in beeves or sheep, it shall be perfect to be accepted; there shall be no blemish therein."],
    ['Leviticus','Lev','22','022','22',"Blind, or broken, or maimed, or having a wen, or scurvy, or scabbed, ye shall not offer these unto the LORD, nor make an offering by fire of them upon the altar unto the LORD."],
    ['Leviticus','Lev','22','023','23',"Either a bullock or a lamb that hath any thing superfluous or lacking in his parts, that mayest thou offer \emph{for} a freewill offering; but for a vow it shall not be accepted."],
    ['Leviticus','Lev','22','024','24',"Ye shall not offer unto the LORD that which is bruised, or crushed, or broken, or cut; neither shall ye make \emph{any} \emph{offering} \emph{thereof} in your land."],
    ['Leviticus','Lev','22','025','25',"Neither from a stranger's hand shall ye offer the bread of your God of any of these; because their corruption \emph{is} in them, \emph{and} blemishes \emph{be} in them: they shall not be accepted for you."],
    ['Leviticus','Lev','22','026','26',"And the LORD spake unto Moses, saying,"],
    ['Leviticus','Lev','22','027','27',"When a bullock, or a sheep, or a goat, is brought forth, then it shall be seven days under the dam; and from the eighth day and thenceforth it shall be accepted for an offering made by fire unto the LORD."],
    ['Leviticus','Lev','22','028','28',"And \emph{whether} \emph{it} \emph{be} cow or ewe, ye shall not kill it and her young both in one day."],
    ['Leviticus','Lev','22','029','29',"And when ye will offer a sacrifice of thanksgiving unto the LORD, offer \emph{it} at your own will."],
    ['Leviticus','Lev','22','030','30',"On the same day it shall be eaten up; ye shall leave none of it until the morrow: I \emph{am} the LORD."],
    ['Leviticus','Lev','22','031','31',"Therefore shall ye keep my commandments, and do them: I \emph{am} the LORD."],
    ['Leviticus','Lev','22','032','32',"Neither shall ye profane my holy name; but I will be hallowed among the children of Israel: I \emph{am} the LORD which hallow you,"],
    ['Leviticus','Lev','22','033','33',"That brought you out of the land of Egypt, to be your God: I \emph{am} the LORD."]
]

Leviticus23 = [
    ['Leviticus','Lev','23','001','1',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','23','002','2',"Speak unto the children of Israel, and say unto them, \emph{Concerning} the feasts of the LORD, which ye shall proclaim \emph{to} \emph{be} holy convocations, \emph{even} these \emph{are} my feasts."],
['Leviticus','Lev','23','003','3',"Six days shall work be done: but the seventh day \emph{is} the sabbath of rest, an holy convocation; ye shall do no work \emph{therein}: it \emph{is} the sabbath of the LORD in all your dwellings."],
['Leviticus','Lev','23','004','4',"These \emph{are} the feasts of the LORD, \emph{even} holy convocations, which ye shall proclaim in their seasons."],
['Leviticus','Lev','23','005','5',"In the fourteenth \emph{day} of the first month at even \emph{is} the LORD'S passover."],
['Leviticus','Lev','23','006','6',"And on the fifteenth day of the same month \emph{is} the feast of unleavened bread unto the LORD: seven days ye must eat unleavened bread."],
['Leviticus','Lev','23','007','7',"In the first day ye shall have an holy convocation: ye shall do no servile work therein."],
['Leviticus','Lev','23','008','8',"But ye shall offer an offering made by fire unto the LORD seven days: in the seventh day \emph{is} an holy convocation: ye shall do no servile work \emph{therein}."],
['Leviticus','Lev','23','009','9',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','23','010','10',"Speak unto the children of Israel, and say unto them, When ye be come into the land which I give unto you, and shall reap the harvest thereof, then ye shall bring a sheaf of the firstfruits of your harvest unto the priest:"],
['Leviticus','Lev','23','011','11',"And he shall wave the sheaf before the LORD, to be accepted for you: on the morrow after the sabbath the priest shall wave it."],
['Leviticus','Lev','23','012','12',"And ye shall offer that day when ye wave the sheaf an he lamb without blemish of the first year for a burnt offering unto the LORD."],
['Leviticus','Lev','23','013','13',"And the meat offering thereof \emph{shall} \emph{be} two tenth deals of fine flour mingled with oil, an offering made by fire unto the LORD \emph{for} a sweet savour: and the drink offering thereof \emph{shall} \emph{be} of wine, the fourth \emph{part} of an hin."],
['Leviticus','Lev','23','014','14',"And ye shall eat neither bread, nor parched corn, nor green ears, until the selfsame day that ye have brought an offering unto your God: \emph{it} \emph{shall} \emph{be} a statute for ever throughout your generations in all your dwellings."],
['Leviticus','Lev','23','015','15',"And ye shall count unto you from the morrow after the sabbath, from the day that ye brought the sheaf of the wave offering; seven sabbaths shall be complete:"],
['Leviticus','Lev','23','016','16',"Even unto the morrow after the seventh sabbath shall ye number fifty days; and ye shall offer a new meat offering unto the LORD."],
['Leviticus','Lev','23','017','17',"Ye shall bring out of your habitations two wave loaves of two tenth deals: they shall be of fine flour; they shall be baken with leaven; \emph{they} \emph{are} the firstfruits unto the LORD."],
['Leviticus','Lev','23','018','18',"And ye shall offer with the bread seven lambs without blemish of the first year, and one young bullock, and two rams: they shall be \emph{for} a burnt offering unto the LORD, with their meat offering, and their drink offerings, \emph{even} an offering made by fire, of sweet savour unto the LORD."],
['Leviticus','Lev','23','019','19',"Then ye shall sacrifice one kid of the goats for a sin offering, and two lambs of the first year for a sacrifice of peace offerings."],
['Leviticus','Lev','23','020','20',"And the priest shall wave them with the bread of the firstfruits \emph{for} a wave offering before the LORD, with the two lambs: they shall be holy to the LORD for the priest."],
['Leviticus','Lev','23','021','21',"And ye shall proclaim on the selfsame day, \emph{that} it may be an holy convocation unto you: ye shall do no servile work \emph{therein:} \emph{it} \emph{shall} \emph{be} a statute for ever in all your dwellings throughout your generations."],
['Leviticus','Lev','23','022','22',"And when ye reap the harvest of your land, thou shalt not make clean riddance of the corners of thy field when thou reapest, neither shalt thou gather any gleaning of thy harvest: thou shalt leave them unto the poor, and to the stranger: I \emph{am} the LORD your God."],
['Leviticus','Lev','23','023','23',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','23','024','24',"Speak unto the children of Israel, saying, In the seventh month, in the first \emph{day} of the month, shall ye have a sabbath, a memorial of blowing of trumpets, an holy convocation."],
['Leviticus','Lev','23','025','25',"Ye shall do no servile work \emph{therein}: but ye shall offer an offering made by fire unto the LORD."],
['Leviticus','Lev','23','026','26',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','23','027','27',"Also on the tenth \emph{day} of this seventh month \emph{there} \emph{shall} \emph{be} a day of atonement: it shall be an holy convocation unto you; and ye shall afflict your souls, and offer an offering made by fire unto the LORD."],
['Leviticus','Lev','23','028','28',"And ye shall do no work in that same day: for it \emph{is} a day of atonement, to make an atonement for you before the LORD your God."],
['Leviticus','Lev','23','029','29',"For whatsoever soul \emph{it} \emph{be} that shall not be afflicted in that same day, he shall be cut off from among his people."],
['Leviticus','Lev','23','030','30',"And whatsoever soul \emph{it} \emph{be} that doeth any work in that same day, the same soul will I destroy from among his people."],
['Leviticus','Lev','23','031','31',"Ye shall do no manner of work: \emph{it} \emph{shall} \emph{be} a statute for ever throughout your generations in all your dwellings."],
['Leviticus','Lev','23','032','32',"It \emph{shall} \emph{be} unto you a sabbath of rest, and ye shall afflict your souls: in the ninth \emph{day} of the month at even, from even unto even, shall ye celebrate your sabbath."],
['Leviticus','Lev','23','033','33',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','23','034','34',"Speak unto the children of Israel, saying, The fifteenth day of this seventh month \emph{shall} \emph{be} the feast of tabernacles \emph{for} seven days unto the LORD."],
['Leviticus','Lev','23','035','35',"On the first day \emph{shall} \emph{be} an holy convocation: ye shall do no servile work \emph{therein}."],
['Leviticus','Lev','23','036','36',"Seven days ye shall offer an offering made by fire unto the LORD: on the eighth day shall be an holy convocation unto you; and ye shall offer an offering made by fire unto the LORD: it \emph{is} a solemn assembly; \emph{and} ye shall do no servile work \emph{therein}."],
['Leviticus','Lev','23','037','37',"These \emph{are} the feasts of the LORD, which ye shall proclaim \emph{to} \emph{be} holy convocations, to offer an offering made by fire unto the LORD, a burnt offering, and a meat offering, a sacrifice, and drink offerings, every thing upon his day:"],
['Leviticus','Lev','23','038','38',"Beside the sabbaths of the LORD, and beside your gifts, and beside all your vows, and beside all your freewill offerings, which ye give unto the LORD."],
['Leviticus','Lev','23','039','39',"Also in the fifteenth day of the seventh month, when ye have gathered in the fruit of the land, ye shall keep a feast unto the LORD seven days: on the first day \emph{shall} \emph{be} a sabbath, and on the eighth day \emph{shall} \emph{be} a sabbath."],
['Leviticus','Lev','23','040','40',"And ye shall take you on the first day the boughs of goodly trees, branches of palm trees, and the boughs of thick trees, and willows of the brook; and ye shall rejoice before the LORD your God seven days."],
['Leviticus','Lev','23','041','41',"And ye shall keep it a feast unto the LORD seven days in the year. \emph{It} \emph{shall} \emph{be} a statute for ever in your generations: ye shall celebrate it in the seventh month."],
['Leviticus','Lev','23','042','42',"Ye shall dwell in booths seven days; all that are Israelites born shall dwell in booths:"],
['Leviticus','Lev','23','043','43',"That your generations may know that I made the children of Israel to dwell in booths, when I brought them out of the land of Egypt: I \emph{am} the LORD your God."],
['Leviticus','Lev','23','044','44',"And Moses declared unto the children of Israel the feasts of the LORD."]
]

Leviticus24 = [
    ['Leviticus','Lev','24','001','1',"And the LORD spake unto Moses, saying,"],
    ['Leviticus','Lev','24','002','2',"Command the children of Israel, that they bring unto thee pure oil olive beaten for the light, to cause the lamps to burn continually."],
    ['Leviticus','Lev','24','003','3',"Without the vail of the testimony, in the tabernacle of the congregation, shall Aaron order it from the evening unto the morning before the LORD continually: \emph{it} \emph{shall} \emph{be} a statute for ever in your generations."],
    ['Leviticus','Lev','24','004','4',"He shall order the lamps upon the pure candlestick before the LORD continually."],
    ['Leviticus','Lev','24','005','5',"And thou shalt take fine flour, and bake twelve cakes thereof: two tenth deals shall be in one cake."],
    ['Leviticus','Lev','24','006','6',"And thou shalt set them in two rows, six on a row, upon the pure table before the LORD."],
    ['Leviticus','Lev','24','007','7',"And thou shalt put pure frankincense upon \emph{each} row, that it may be on the bread for a memorial, \emph{even} an offering made by fire unto the LORD."],
    ['Leviticus','Lev','24','008','8',"Every sabbath he shall set it in order before the LORD continually, \emph{being} \emph{taken} from the children of Israel by an everlasting covenant."],
    ['Leviticus','Lev','24','009','9',"And it shall be Aaron's and his sons'; and they shall eat it in the holy place: for it \emph{is} most holy unto him of the offerings of the LORD made by fire by a perpetual statute."],
    ['Leviticus','Lev','24','010','10',"And the son of an Israelitish woman, whose father \emph{was} an Egyptian, went out among the children of Israel: and this son of the Israelitish \emph{woman} and a man of Israel strove together in the camp;"],
    ['Leviticus','Lev','24','011','11',"And the Israelitish woman's son blasphemed the name \emph{of} \emph{the} \emph{LORD}, and cursed. And they brought him unto Moses: (and his mother's name \emph{was} Shelomith, the daughter of Dibri, of the tribe of Dan:)"],
    ['Leviticus','Lev','24','012','12',"And they put him in ward, that the mind of the LORD might be shewed them."],
    ['Leviticus','Lev','24','013','13',"And the LORD spake unto Moses, saying,"],
    ['Leviticus','Lev','24','014','14',"Bring forth him that hath cursed without the camp; and let all that heard \emph{him} lay their hands upon his head, and let all the congregation stone him."],
    ['Leviticus','Lev','24','015','15',"And thou shalt speak unto the children of Israel, saying, Whosoever curseth his God shall bear his sin."],
    ['Leviticus','Lev','24','016','16',"And he that blasphemeth the name of the LORD, he shall surely be put to death, \emph{and} all the congregation shall certainly stone him: as well the stranger, as he that is born in the land, when he blasphemeth the name \emph{of} \emph{the} \emph{LORD}, shall be put to death."],
    ['Leviticus','Lev','24','017','17',"And he that killeth any man shall surely be put to death."],
    ['Leviticus','Lev','24','018','18',"And he that killeth a beast shall make it good; beast for beast."],
    ['Leviticus','Lev','24','019','19',"And if a man cause a blemish in his neighbour; as he hath done, so shall it be done to him;"],
    ['Leviticus','Lev','24','020','20',"Breach for breach, eye for eye, tooth for tooth: as he hath caused a blemish in a man, so shall it be done to him \emph{again}."],
    ['Leviticus','Lev','24','021','21',"And he that killeth a beast, he shall restore it: and he that killeth a man, he shall be put to death."],
    ['Leviticus','Lev','24','022','22',"Ye shall have one manner of law, as well for the stranger, as for one of your own country: for I \emph{am} the LORD your God."],
    ['Leviticus','Lev','24','023','23',"And Moses spake to the children of Israel, that they should bring forth him that had cursed out of the camp, and stone him with stones. And the children of Israel did as the LORD commanded Moses."]
]

Leviticus25 = [
    ['Leviticus','Lev','25','001','1',"And the LORD spake unto Moses in mount Sinai, saying,"],
['Leviticus','Lev','25','002','2',"Speak unto the children of Israel, and say unto them, When ye come into the land which I give you, then shall the land keep a sabbath unto the LORD."],
['Leviticus','Lev','25','003','3',"Six years thou shalt sow thy field, and six years thou shalt prune thy vineyard, and gather in the fruit thereof;"],
['Leviticus','Lev','25','004','4',"But in the seventh year shall be a sabbath of rest unto the land, a sabbath for the LORD: thou shalt neither sow thy field, nor prune thy vineyard."],
['Leviticus','Lev','25','005','5',"That which groweth of its own accord of thy harvest thou shalt not reap, neither gather the grapes of thy vine undressed: \emph{for} it is a year of rest unto the land."],
['Leviticus','Lev','25','006','6',"And the sabbath of the land shall be meat for you; for thee, and for thy servant, and for thy maid, and for thy hired servant, and for thy stranger that sojourneth with thee,"],
['Leviticus','Lev','25','007','7',"And for thy cattle, and for the beast that \emph{are} in thy land, shall all the increase thereof be meat."],
['Leviticus','Lev','25','008','8',"And thou shalt number seven sabbaths of years unto thee, seven times seven years; and the space of the seven sabbaths of years shall be unto thee forty and nine years."],
['Leviticus','Lev','25','009','9',"Then shalt thou cause the trumpet of the jubile to sound on the tenth \emph{day} of the seventh month, in the day of atonement shall ye make the trumpet sound throughout all your land."],
['Leviticus','Lev','25','010','10',"And ye shall hallow the fiftieth year, and proclaim liberty throughout \emph{all} the land unto all the inhabitants thereof: it shall be a jubile unto you; and ye shall return every man unto his possession, and ye shall return every man unto his family."],
['Leviticus','Lev','25','011','11',"A jubile shall that fiftieth year be unto you: ye shall not sow, neither reap that which groweth of itself in it, nor gather \emph{the} \emph{grapes} in it of thy vine undressed."],
['Leviticus','Lev','25','012','12',"For it \emph{is} the jubile; it shall be holy unto you: ye shall eat the increase thereof out of the field."],
['Leviticus','Lev','25','013','13',"In the year of this jubile ye shall return every man unto his possession."],
['Leviticus','Lev','25','014','14',"And if thou sell ought unto thy neighbour, or buyest \emph{ought} of thy neighbour's hand, ye shall not oppress one another:"],
['Leviticus','Lev','25','015','15',"According to the number of years after the jubile thou shalt buy of thy neighbour, \emph{and} according unto the number of years of the fruits he shall sell unto thee:"],
['Leviticus','Lev','25','016','16',"According to the multitude of years thou shalt increase the price thereof, and according to the fewness of years thou shalt diminish the price of it: for \emph{according} to the number \emph{of} \emph{the} \emph{years} of the fruits doth he sell unto thee."],
['Leviticus','Lev','25','017','17',"Ye shall not therefore oppress one another; but thou shalt fear thy God: for I \emph{am} the LORD your God."],
['Leviticus','Lev','25','018','18',"Wherefore ye shall do my statutes, and keep my judgments, and do them; and ye shall dwell in the land in safety."],
['Leviticus','Lev','25','019','19',"And the land shall yield her fruit, and ye shall eat your fill, and dwell therein in safety."],
['Leviticus','Lev','25','020','20',"And if ye shall say, What shall we eat the seventh year? behold, we shall not sow, nor gather in our increase:"],
['Leviticus','Lev','25','021','21',"Then I will command my blessing upon you in the sixth year, and it shall bring forth fruit for three years."],
['Leviticus','Lev','25','022','22',"And ye shall sow the eighth year, and eat \emph{yet} of old fruit until the ninth year; until her fruits come in ye shall eat \emph{of} the old \emph{store}."],
['Leviticus','Lev','25','023','23',"The land shall not be sold for ever: for the land \emph{is} mine; for ye \emph{are} strangers and sojourners with me."],
['Leviticus','Lev','25','024','24',"And in all the land of your possession ye shall grant a redemption for the land."],
['Leviticus','Lev','25','025','25',"If thy brother be waxen poor, and hath sold away \emph{some} of his possession, and if any of his kin come to redeem it, then shall he redeem that which his brother sold."],
['Leviticus','Lev','25','026','26',"And if the man have none to redeem it, and himself be able to redeem it;"],
['Leviticus','Lev','25','027','27',"Then let him count the years of the sale thereof, and restore the overplus unto the man to whom he sold it; that he may return unto his possession."],
['Leviticus','Lev','25','028','28',"But if he be not able to restore \emph{it} to him, then that which is sold shall remain in the hand of him that hath bought it until the year of jubile: and in the jubile it shall go out, and he shall return unto his possession."],
['Leviticus','Lev','25','029','29',"And if a man sell a dwelling house in a walled city, then he may redeem it within a whole year after it is sold; \emph{within} a full year may he redeem it."],
['Leviticus','Lev','25','030','30',"And if it be not redeemed within the space of a full year, then the house that \emph{is} in the walled city shall be established for ever to him that bought it throughout his generations: it shall not go out in the jubile."],
['Leviticus','Lev','25','031','31',"But the houses of the villages which have no wall round about them shall be counted as the fields of the country: they may be redeemed, and they shall go out in the jubile."],
['Leviticus','Lev','25','032','32',"Notwithstanding the cities of the Levites, \emph{and} the houses of the cities of their possession, may the Levites redeem at any time."],
['Leviticus','Lev','25','033','33',"And if a man purchase of the Levites, then the house that was sold, and the city of his possession, shall go out in \emph{the} \emph{year} \emph{of} jubile: for the houses of the cities of the Levites \emph{are} their possession among the children of Israel."],
['Leviticus','Lev','25','034','34',"But the field of the suburbs of their cities may not be sold; for it \emph{is} their perpetual possession."],
['Leviticus','Lev','25','035','35',"And if thy brother be waxen poor, and fallen in decay with thee; then thou shalt relieve him: \emph{yea,} \emph{though} \emph{he} \emph{be} a stranger, or a sojourner; that he may live with thee."],
['Leviticus','Lev','25','036','36',"Take thou no usury of him, or increase: but fear thy God; that thy brother may live with thee."],
['Leviticus','Lev','25','037','37',"Thou shalt not give him thy money upon usury, nor lend him thy victuals for increase."],
['Leviticus','Lev','25','038','38',"I \emph{am} the LORD your God, which brought you forth out of the land of Egypt, to give you the land of Canaan, \emph{and} to be your God."],
['Leviticus','Lev','25','039','39',"And if thy brother \emph{that} \emph{dwelleth} by thee be waxen poor, and be sold unto thee; thou shalt not compel him to serve as a bondservant:"],
['Leviticus','Lev','25','040','40',"\emph{But} as an hired servant, \emph{and} as a sojourner, he shall be with thee, \emph{and} shall serve thee unto the year of jubile:"],
['Leviticus','Lev','25','041','41',"And \emph{then} shall he depart from thee, \emph{both} he and his children with him, and shall return unto his own family, and unto the possession of his fathers shall he return."],
['Leviticus','Lev','25','042','42',"For they \emph{are} my servants, which I brought forth out of the land of Egypt: they shall not be sold as bondmen."],
['Leviticus','Lev','25','043','43',"Thou shalt not rule over him with rigour; but shalt fear thy God."],
['Leviticus','Lev','25','044','44',"Both thy bondmen, and thy bondmaids, which thou shalt have, \emph{shall} \emph{be} of the heathen that are round about you; of them shall ye buy bondmen and bondmaids."],
['Leviticus','Lev','25','045','45',"Moreover of the children of the strangers that do sojourn among you, of them shall ye buy, and of their families that \emph{are} with you, which they begat in your land: and they shall be your possession."],
['Leviticus','Lev','25','046','46',"And ye shall take them as an inheritance for your children after you, to inherit \emph{them} \emph{for} a possession; they shall be your bondmen for ever: but over your brethren the children of Israel, ye shall not rule one over another with rigour."],
['Leviticus','Lev','25','047','47',"And if a sojourner or stranger wax rich by thee, and thy brother \emph{that} \emph{dwelleth} by him wax poor, and sell himself unto the stranger \emph{or} sojourner by thee, or to the stock of the stranger's family:"],
['Leviticus','Lev','25','048','48',"After that he is sold he may be redeemed again; one of his brethren may redeem him:"],
['Leviticus','Lev','25','049','49',"Either his uncle, or his uncle's son, may redeem him, or \emph{any} that is nigh of kin unto him of his family may redeem him; or if he be able, he may redeem himself."],
['Leviticus','Lev','25','050','50',"And he shall reckon with him that bought him from the year that he was sold to him unto the year of jubile: and the price of his sale shall be according unto the number of years, according to the time of an hired servant shall it be with him."],
['Leviticus','Lev','25','051','51',"If \emph{there} \emph{be} yet many years \emph{behind}, according unto them he shall give again the price of his redemption out of the money that he was bought for."],
['Leviticus','Lev','25','052','52',"And if there remain but few years unto the year of jubile, then he shall count with him, \emph{and} according unto his years shall he give him again the price of his redemption."],
['Leviticus','Lev','25','053','53',"\emph{And} as a yearly hired servant shall he be with him: \emph{and} \emph{the} \emph{other} shall not rule with rigour over him in thy sight."],
['Leviticus','Lev','25','054','54',"And if he be not redeemed in these \emph{years}, then he shall go out in the year of jubile, \emph{both} he, and his children with him."],
['Leviticus','Lev','25','055','55',"For unto me the children of Israel \emph{are} servants; they \emph{are} my servants whom I brought forth out of the land of Egypt: I \emph{am} the LORD your God."]
   ]

Leviticus26 = [
    ['Leviticus','Lev','26','001','1',"Ye shall make you no idols nor graven image, neither rear you up a standing image, neither shall ye set up \emph{any} image of stone in your land, to bow down unto it: for I \emph{am} the LORD your God."],
['Leviticus','Lev','26','002','2',"Ye shall keep my sabbaths, and reverence my sanctuary: I \emph{am} the LORD."],
['Leviticus','Lev','26','003','3',"If ye walk in my statutes, and keep my commandments, and do them;"],
['Leviticus','Lev','26','004','4',"Then I will give you rain in due season, and the land shall yield her increase, and the trees of the field shall yield their fruit."],
['Leviticus','Lev','26','005','5',"And your threshing shall reach unto the vintage, and the vintage shall reach unto the sowing time: and ye shall eat your bread to the full, and dwell in your land safely."],
['Leviticus','Lev','26','006','6',"And I will give peace in the land, and ye shall lie down, and none shall make \emph{you} afraid: and I will rid evil beasts out of the land, neither shall the sword go through your land."],
['Leviticus','Lev','26','007','7',"And ye shall chase your enemies, and they shall fall before you by the sword."],
['Leviticus','Lev','26','008','8',"And five of you shall chase an hundred, and an hundred of you shall put ten thousand to flight: and your enemies shall fall before you by the sword."],
['Leviticus','Lev','26','009','9',"For I will have respect unto you, and make you fruitful, and multiply you, and establish my covenant with you."],
['Leviticus','Lev','26','010','10',"And ye shall eat old store, and bring forth the old because of the new."],
['Leviticus','Lev','26','011','11',"And I will set my tabernacle among you: and my soul shall not abhor you."],
['Leviticus','Lev','26','012','12',"And I will walk among you, and will be your God, and ye shall be my people."],
['Leviticus','Lev','26','013','13',"I \emph{am} the LORD your God, which brought you forth out of the land of Egypt, that ye should not be their bondmen; and I have broken the bands of your yoke, and made you go upright."],
['Leviticus','Lev','26','014','14',"But if ye will not hearken unto me, and will not do all these commandments;"],
['Leviticus','Lev','26','015','15',"And if ye shall despise my statutes, or if your soul abhor my judgments, so that ye will not do all my commandments, \emph{but} that ye break my covenant:"],
['Leviticus','Lev','26','016','16',"I also will do this unto you; I will even appoint over you terror, consumption, and the burning ague, that shall consume the eyes, and cause sorrow of heart: and ye shall sow your seed in vain, for your enemies shall eat it."],
['Leviticus','Lev','26','017','17',"And I will set my face against you, and ye shall be slain before your enemies: they that hate you shall reign over you; and ye shall flee when none pursueth you."],
['Leviticus','Lev','26','018','18',"And if ye will not yet for all this hearken unto me, then I will punish you seven times more for your sins."],
['Leviticus','Lev','26','019','19',"And I will break the pride of your power; and I will make your heaven as iron, and your earth as brass:"],
['Leviticus','Lev','26','020','20',"And your strength shall be spent in vain: for your land shall not yield her increase, neither shall the trees of the land yield their fruits."],
['Leviticus','Lev','26','021','21',"And if ye walk contrary unto me, and will not hearken unto me; I will bring seven times more plagues upon you according to your sins."],
['Leviticus','Lev','26','022','22',"I will also send wild beasts among you, which shall rob you of your children, and destroy your cattle, and make you few in number; and your \emph{high} ways shall be desolate."],
['Leviticus','Lev','26','023','23',"And if ye will not be reformed by me by these things, but will walk contrary unto me;"],
['Leviticus','Lev','26','024','24',"Then will I also walk contrary unto you, and will punish you yet seven times for your sins."],
['Leviticus','Lev','26','025','25',"And I will bring a sword upon you, that shall avenge the quarrel of \emph{my} covenant: and when ye are gathered together within your cities, I will send the pestilence among you; and ye shall be delivered into the hand of the enemy."],
['Leviticus','Lev','26','026','26',"\emph{And} when I have broken the staff of your bread, ten women shall bake your bread in one oven, and they shall deliver \emph{you} your bread again by weight: and ye shall eat, and not be satisfied."],
['Leviticus','Lev','26','027','27',"And if ye will not for all this hearken unto me, but walk contrary unto me;"],
['Leviticus','Lev','26','028','28',"Then I will walk contrary unto you also in fury; and I, even I, will chastise you seven times for your sins."],
['Leviticus','Lev','26','029','29',"And ye shall eat the flesh of your sons, and the flesh of your daughters shall ye eat."],
['Leviticus','Lev','26','030','30',"And I will destroy your high places, and cut down your images, and cast your carcases upon the carcases of your idols, and my soul shall abhor you."],
['Leviticus','Lev','26','031','31',"And I will make your cities waste, and bring your sanctuaries unto desolation, and I will not smell the savour of your sweet odours."],
['Leviticus','Lev','26','032','32',"And I will bring the land into desolation: and your enemies which dwell therein shall be astonished at it."],
['Leviticus','Lev','26','033','33',"And I will scatter you among the heathen, and will draw out a sword after you: and your land shall be desolate, and your cities waste."],
['Leviticus','Lev','26','034','34',"Then shall the land enjoy her sabbaths, as long as it lieth desolate, and ye \emph{be} in your enemies' land; \emph{even} then shall the land rest, and enjoy her sabbaths."],
['Leviticus','Lev','26','035','35',"As long as it lieth desolate it shall rest; because it did not rest in your sabbaths, when ye dwelt upon it."],
['Leviticus','Lev','26','036','36',"And upon them that are left \emph{alive} of you I will send a faintness into their hearts in the lands of their enemies; and the sound of a shaken leaf shall chase them; and they shall flee, as fleeing from a sword; and they shall fall when none pursueth."],
['Leviticus','Lev','26','037','37',"And they shall fall one upon another, as it were before a sword, when none pursueth: and ye shall have no power to stand before your enemies."],
['Leviticus','Lev','26','038','38',"And ye shall perish among the heathen, and the land of your enemies shall eat you up."],
['Leviticus','Lev','26','039','39',"And they that are left of you shall pine away in their iniquity in your enemies' lands; and also in the iniquities of their fathers shall they pine away with them."],
['Leviticus','Lev','26','040','40',"If they shall confess their iniquity, and the iniquity of their fathers, with their trespass which they trespassed against me, and that also they have walked contrary unto me;"],
['Leviticus','Lev','26','041','41',"And \emph{that} I also have walked contrary unto them, and have brought them into the land of their enemies; if then their uncircumcised hearts be humbled, and they then accept of the punishment of their iniquity:"],
['Leviticus','Lev','26','042','42',"Then will I remember my covenant with Jacob, and also my covenant with Isaac, and also my covenant with Abraham will I remember; and I will remember the land."],
['Leviticus','Lev','26','043','43',"The land also shall be left of them, and shall enjoy her sabbaths, while she lieth desolate without them: and they shall accept of the punishment of their iniquity: because, even because they despised my judgments, and because their soul abhorred my statutes."],
['Leviticus','Lev','26','044','44',"And yet for all that, when they be in the land of their enemies, I will not cast them away, neither will I abhor them, to destroy them utterly, and to break my covenant with them: for I \emph{am} the LORD their God."],
['Leviticus','Lev','26','045','45',"But I will for their sakes remember the covenant of their ancestors, whom I brought forth out of the land of Egypt in the sight of the heathen, that I might be their God: I \emph{am} the LORD."],
['Leviticus','Lev','26','046','46',"These \emph{are} the statutes and judgments and laws, which the LORD made between him and the children of Israel in mount Sinai by the hand of Moses."]
]

Leviticus27 = [
    ['Leviticus','Lev','27','001','1',"And the LORD spake unto Moses, saying,"],
['Leviticus','Lev','27','002','2',"Speak unto the children of Israel, and say unto them, When a man shall make a singular vow, the persons \emph{shall} \emph{be} for the LORD by thy estimation."],
['Leviticus','Lev','27','003','3',"And thy estimation shall be of the male from twenty years old even unto sixty years old, even thy estimation shall be fifty shekels of silver, after the shekel of the sanctuary."],
['Leviticus','Lev','27','004','4',"And if it \emph{be} a female, then thy estimation shall be thirty shekels."],
['Leviticus','Lev','27','005','5',"And if \emph{it} \emph{be} from five years old even unto twenty years old, then thy estimation shall be of the male twenty shekels, and for the female ten shekels."],
['Leviticus','Lev','27','006','6',"And if \emph{it} \emph{be} from a month old even unto five years old, then thy estimation shall be of the male five shekels of silver, and for the female thy estimation \emph{shall} \emph{be} three shekels of silver."],
['Leviticus','Lev','27','007','7',"And if \emph{it} \emph{be} from sixty years old and above; if \emph{it} \emph{be} a male, then thy estimation shall be fifteen shekels, and for the female ten shekels."],
['Leviticus','Lev','27','008','8',"But if he be poorer than thy estimation, then he shall present himself before the priest, and the priest shall value him; according to his ability that vowed shall the priest value him."],
['Leviticus','Lev','27','009','9',"And if \emph{it} \emph{be} a beast, whereof men bring an offering unto the LORD, all that \emph{any} \emph{man} giveth of such unto the LORD shall be holy."],
['Leviticus','Lev','27','010','10',"He shall not alter it, nor change it, a good for a bad, or a bad for a good: and if he shall at all change beast for beast, then it and the exchange thereof shall be holy."],
['Leviticus','Lev','27','011','11',"And if \emph{it} \emph{be} any unclean beast, of which they do not offer a sacrifice unto the LORD, then he shall present the beast before the priest:"],
['Leviticus','Lev','27','012','12',"And the priest shall value it, whether it be good or bad: as thou valuest it, \emph{who} \emph{art} the priest, so shall it be."],
['Leviticus','Lev','27','013','13',"But if he will at all redeem it, then he shall add a fifth \emph{part} thereof unto thy estimation."],
['Leviticus','Lev','27','014','14',"And when a man shall sanctify his house \emph{to} \emph{be} holy unto the LORD, then the priest shall estimate it, whether it be good or bad: as the priest shall estimate it, so shall it stand."],
['Leviticus','Lev','27','015','15',"And if he that sanctified it will redeem his house, then he shall add the fifth \emph{part} of the money of thy estimation unto it, and it shall be his."],
['Leviticus','Lev','27','016','16',"And if a man shall sanctify unto the LORD \emph{some} \emph{part} of a field of his possession, then thy estimation shall be according to the seed thereof: an homer of barley seed \emph{shall} \emph{be} \emph{valued} at fifty shekels of silver."],
['Leviticus','Lev','27','017','17',"If he sanctify his field from the year of jubile, according to thy estimation it shall stand."],
['Leviticus','Lev','27','018','18',"But if he sanctify his field after the jubile, then the priest shall reckon unto him the money according to the years that remain, even unto the year of the jubile, and it shall be abated from thy estimation."],
['Leviticus','Lev','27','019','19',"And if he that sanctified the field will in any wise redeem it, then he shall add the fifth \emph{part} of the money of thy estimation unto it, and it shall be assured to him."],
['Leviticus','Lev','27','020','20',"And if he will not redeem the field, or if he have sold the field to another man, it shall not be redeemed any more."],
['Leviticus','Lev','27','021','21',"But the field, when it goeth out in the jubile, shall be holy unto the LORD, as a field devoted; the possession thereof shall be the priest's."],
['Leviticus','Lev','27','022','22',"And if \emph{a} \emph{man} sanctify unto the LORD a field which he hath bought, which \emph{is} not of the fields of his possession;"],
['Leviticus','Lev','27','023','23',"Then the priest shall reckon unto him the worth of thy estimation, \emph{even} unto the year of the jubile: and he shall give thine estimation in that day, \emph{as} a holy thing unto the LORD."],
['Leviticus','Lev','27','024','24',"In the year of the jubile the field shall return unto him of whom it was bought, \emph{even} to him to whom the possession of the land \emph{did} \emph{belong}."],
['Leviticus','Lev','27','025','25',"And all thy estimations shall be according to the shekel of the sanctuary: twenty gerahs shall be the shekel."],
['Leviticus','Lev','27','026','26',"Only the firstling of the beasts, which should be the LORD'S firstling, no man shall sanctify it; whether \emph{it} \emph{be} ox, or sheep: it \emph{is} the LORD'S."],
['Leviticus','Lev','27','027','27',"And if \emph{it} \emph{be} of an unclean beast, then he shall redeem \emph{it} according to thine estimation, and shall add a fifth \emph{part} of it thereto: or if it be not redeemed, then it shall be sold according to thy estimation."],
['Leviticus','Lev','27','028','28',"Notwithstanding no devoted thing, that a man shall devote unto the LORD of all that he hath, \emph{both} of man and beast, and of the field of his possession, shall be sold or redeemed: every devoted thing \emph{is} most holy unto the LORD."],
['Leviticus','Lev','27','029','29',"None devoted, which shall be devoted of men, shall be redeemed; \emph{but} shall surely be put to death."],
['Leviticus','Lev','27','030','30',"And all the tithe of the land, \emph{whether} of the seed of the land, \emph{or} of the fruit of the tree, \emph{is} the LORD'S: \emph{it} \emph{is} holy unto the LORD."],
['Leviticus','Lev','27','031','31',"And if a man will at all redeem \emph{ought} of his tithes, he shall add thereto the fifth \emph{part} thereof."],
['Leviticus','Lev','27','032','32',"And concerning the tithe of the herd, or of the flock, \emph{even} of whatsoever passeth under the rod, the tenth shall be holy unto the LORD."],
['Leviticus','Lev','27','033','33',"He shall not search whether it be good or bad, neither shall he change it: and if he change it at all, then both it and the change thereof shall be holy; it shall not be redeemed."],
['Leviticus','Lev','27','034','34',"These \emph{are} the commandments, which the LORD commanded Moses for the children of Israel in mount Sinai."]
]

def main():
    ProcessVerses('Leviticus26', Leviticus26)

if __name__ == "__main__":
    main()
